In [0]:
import pandas as pd
import sklearn.model_selection
import sklearn.ensemble

#gf
import mlflow
import mlflow.keras
import mlflow.tensorflow
from hyperopt import fmin, tpe, hp, SparkTrials, Trials, STATUS_OK
from hyperopt.pyll import scope

from tensorflow.keras.layers import Dense, LSTM
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Normalization
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR

#import streamlit as st
import joblib

import tensorflow as tf
#Librerias para redes neuronales(secuencial)
from tensorflow.keras.models import Sequential
#Capa completamente conectada
from tensorflow.keras.layers import Dense, Normalization
#Optimizador
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler

# Configurar pandas para mostrar todas las filas y columnas
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

#### 1. Leer datos para crear los Modelos IA

In [0]:
df_delta = spark.read.format("delta").load("/mnt/datalakemlopsd4m/presentation/proyectopases_presentation/datos_procesados_hudbay_tabladelta")
datos = df_delta.toPandas()
datos.head()

In [0]:
datos.dtypes

Veamos el periodo de datos que estaremos utilizando

In [0]:
datos['tiempo_inicio_carga_carguio'].min(), datos['tiempo_inicio_carga_carguio'].max(),

Veamos la dimension de los datos a utilizar

In [0]:
datos.shape

#### 2. Hacemos un Tratamiento de variable categorica con One Hot Encoding

##### 2.1 Metodo 1 - Usando la libreria de One Hot Encoder de Sckilearn

In [0]:
from sklearn.preprocessing import OneHotEncoder
import pickle
# Aplicar el One Hot Encoding
encoder = OneHotEncoder(sparse_output=False)   # Saprce para manejar el numero de ceros, en la generacion del one hot encoding
one_hot = encoder.fit_transform(datos[['nombre_equipo_carguio']])

# Guardar el codificador One Hot Encoding entrenado
with open('one_hot_encoder_pases.pkl', 'wb') as f:
    pickle.dump(encoder, f)

# Convertir a DataFrame
one_hot_df = pd.DataFrame(one_hot, columns=encoder.get_feature_names_out(['nombre_equipo_carguio']))  #equipo_carguio , nombre acompanate en la columnas de los dummies

# Unir el DataFrame original con el codificado
datos = pd.concat([datos, one_hot_df], axis=1)

# Mostrar el resultado
datos.head()

In [0]:
# Supongamos que se agrega una nueva fila a la base de datos
nueva_fila = pd.DataFrame({'nombre_equipo_carguio': ['CF03']})

# # Cargar el codificador guardado
with open('one_hot_encoder_pases.pkl', 'rb') as f:
     encoder = pickle.load(f)

# # Transformar la nueva fila
one_hot_nueva_fila = encoder.transform(nueva_fila)

# # Convertir a DataFrame
one_hot_nueva_fila_df = pd.DataFrame(one_hot_nueva_fila, columns=encoder.get_feature_names_out(['nombre_equipo_carguio']))

one_hot_nueva_fila_df  # De aqui filtras las columnas y le vas pasando al Modelo de IA

##### 2.2 Metodo 2 -  Usar un bucle para realizar el codigo de One Hot Encoding

2.2.1 VARIABLE nombre_equipo_carguio

In [0]:
import pandas as pd

# Obtener las categorías únicas
categorias1 = datos['nombre_equipo_carguio'].unique()

# Crear un DataFrame vacío con columnas para cada categoría, Diccionario one_hot_dict : categoria: clave es una categoría única de #nombre_equipo_carguio, y el [0]*len(datos) : valor asociado a cada clave es una lista de ceros de la misma longitud que el DataFrame 
one_hot_dict = {categoria: [0]*len(datos) for categoria in categorias1}

# Llenar el diccionario con 1s en las posiciones correspondientes
# idx es el índice de la fila, val es el valor de nombre_equipo_carguio en esa fila
# Si idx=0 y val='CF02', entonces one_hot_dict['CF02'][0] = 1
for idx, val in datos['nombre_equipo_carguio'].items():
    one_hot_dict[val][idx] = 1

# Convertir el diccionario a un DataFrame
one_hot_df = pd.DataFrame(one_hot_dict)

#Unir el DataFrame original con el codificado
datos = pd.concat([datos, one_hot_df], axis=1)

# # Mostrar el resultado
datos[['nombre_equipo_carguio','PH003', 'CF002', 'CF001', 'PH001', 'PH002']].head()

Aplicar el OneHotEncoding en Produccion VARIABLE nombre_equipo_carguio

In [0]:
import pandas as pd

# Cargar las categorías guardadas
# categorias = pd.read_csv('categorias.csv', header=None).squeeze().tolist()
categorias1 = ['PH003', 'CF002', 'CF001', 'PH001', 'PH002']

def aplicar_one_hot_equipo(nuevo_dato, categorias):
    """
    Realiza el One-Hot Encoding de la columna 'nombre_equipo_carguio' para un nuevo registro.

    Parámetros:
    - nuevo_dato (pd.DataFrame): DataFrame con el nuevo registro.
    - categorias (list): Lista de categorías conocidas para la codificación.

    Retorno:
    - pd.DataFrame: DataFrame original con las columnas de One-Hot Encoding añadidas.
    """
    # Crear un DataFrame vacío para el One-Hot Encoding
    one_hot_dict = {categoria: [0]*len(nuevo_dato) for categoria in categorias}

    # Llenar el diccionario con 1s en las posiciones correspondientes
    for idx, val in nuevo_dato['nombre_equipo_carguio'].items():
        if val in one_hot_dict:
            one_hot_dict[val][idx] = 1
        # Si la categoría no se encuentra, los ceros permanecen

    # Convertir el diccionario a un DataFrame
    one_hot_df = pd.DataFrame(one_hot_dict)

    # Unir el nuevo DataFrame con el DataFrame codificado
    nuevo_dato = pd.concat([nuevo_dato, one_hot_df], axis=1)
    
    return nuevo_dato

In [0]:
nuevo_dato = pd.DataFrame({'nombre_equipo_carguio': ['PH002']})
nuevo_dato

In [0]:
resultado = aplicar_one_hot_equipo(nuevo_dato, categorias1)
print(resultado)

In [0]:
categorias1

2.2.2 VARIABLE id_material_dominante_en_poligono

In [0]:
import pandas as pd

# Obtener las categorías únicas
categorias2 = datos['id_material_dominante_en_poligono'].unique()

# Crear un DataFrame vacío con columnas para cada categoría, Diccionario one_hot_dict : categoria: clave es una categoría única de #nombre_equipo_carguio, y el [0]*len(datos) : valor asociado a cada clave es una lista de ceros de la misma longitud que el DataFrame 
one_hot_dict = {categoria: [0]*len(datos) for categoria in categorias2}

# Llenar el diccionario con 1s en las posiciones correspondientes
# idx es el índice de la fila, val es el valor de nombre_equipo_carguio en esa fila
# Si idx=0 y val='CF02', entonces one_hot_dict['CF02'][0] = 1
for idx, val in datos['id_material_dominante_en_poligono'].items():
    one_hot_dict[val][idx] = 1

# Convertir el diccionario a un DataFrame
one_hot_df2 = pd.DataFrame(one_hot_dict)

#Unir el DataFrame original con el codificado
datos = pd.concat([datos, one_hot_df2], axis=1)

# Diccionario de columnas a renombrar: clave = nombre actual, valor = nuevo nombre
nuevos_nombres = {
    8.:'material_8',
    7.:'material_7',
    1.:'material_1',
    4.:'material_4',
    5.:'material_5',
    9.:'material_9',
    6.:'material_6',
    12.:'material_12',
    13.:'material_13',
    11.:'material_11',
    2.:'material_2',
    15.:'material_15',
    # Asumiendo que hay una columna `col13`, pero no está en el ejemplo, la omitiré
    # 'col13': 13.,  # Esta no existe en el DataFrame actual
    # 'col15': 15.  # Esta tampoco existe en el DataFrame actual
}

# Renombrar las columnas especificadas
datos = datos.rename(columns=nuevos_nombres)

# # Mostrar el resultado
datos[['material_8','material_7','material_1','material_4','material_5','material_9','material_6','material_12','material_13','material_11',
    'material_2','material_15']].head()

In [0]:
import pandas as pd

# Cargar las categorías guardadas
# categorias = pd.read_csv('categorias.csv', header=None).squeeze().tolist()
categorias2 = [ 8.,  7.,  1.,  4.,  5.,  9.,  6., 12., 13., 11.,  2., 15.]

def aplicar_one_hot_material(nuevo_dato, categorias):
    """
    Realiza el One-Hot Encoding de la columna 'nombre_equipo_carguio' para un nuevo registro.

    Parámetros:
    - nuevo_dato (pd.DataFrame): DataFrame con el nuevo registro.
    - categorias (list): Lista de categorías conocidas para la codificación.

    Retorno:
    - pd.DataFrame: DataFrame original con las columnas de One-Hot Encoding añadidas.
    """
    # Crear un DataFrame vacío para el One-Hot Encoding
    one_hot_dict = {categoria: [0]*len(nuevo_dato) for categoria in categorias}

    # Llenar el diccionario con 1s en las posiciones correspondientes
    for idx, val in nuevo_dato['id_material_dominante_en_poligono'].items():
        if val in one_hot_dict:
            one_hot_dict[val][idx] = 1
        # Si la categoría no se encuentra, los ceros permanecen

    # Convertir el diccionario a un DataFrame
    one_hot_df = pd.DataFrame(one_hot_dict)

    #Unir el DataFrame original con el codificado
    nuevo_dato = pd.concat([nuevo_dato, one_hot_df], axis=1)

    # Diccionario de columnas a renombrar: clave = nombre actual, valor = nuevo nombre
    nuevos_nombres = {
    8.:'material_8',
    7.:'material_7',
    1.:'material_1',
    4.:'material_4',
    5.:'material_5',
    9.:'material_9',
    6.:'material_6',
    12.:'material_12',
    13.:'material_13',
    11.:'material_11',
    2.:'material_2',
    15.:'material_15',
    # Asumiendo que hay una columna `col13`, pero no está en el ejemplo, la omitiré
    # 'col13': 13.,  # Esta no existe en el DataFrame actual
    # 'col15': 15.  # Esta tampoco existe en el DataFrame actual
    }

    # Renombrar las columnas especificadas
    nuevo_dato = nuevo_dato.rename(columns=nuevos_nombres)

    # Convertir el diccionario a un DataFrame
    one_hot_df = pd.DataFrame(one_hot_dict)

    # Unir el nuevo DataFrame con el DataFrame codificado
    nuevo_dato = pd.concat([nuevo_dato, one_hot_df], axis=1)
    
    return nuevo_dato

In [0]:
nuevo_dato1 = pd.DataFrame({'id_material_dominante_en_poligono': [8.]})
nuevo_dato1

In [0]:
resultado1 = aplicar_one_hot_material(nuevo_dato1, categorias2)
resultado1

In [0]:
categorias2

VARIABLE Procedencia - Esta variable sera reemplazada por la variable ID_MATERIAL

In [0]:
# Obtener las categorías únicas
categorias_pro = datos['Procedencia'].unique()

# Crear un DataFrame vacío con columnas para cada categoría, Diccionario one_hot_dict : categoria: clave es una categoría única de #nombre_equipo_carguio, y el [0]*len(datos) : valor asociado a cada clave es una lista de ceros de la misma longitud que el DataFrame 
one_hot_dict_pro = {categoria: [0]*len(datos) for categoria in categorias_pro}

# Llenar el diccionario con 1s en las posiciones correspondientes
# idx es el índice de la fila, val es el valor de nombre_equipo_carguio en esa fila
# Si idx=0 y val='CF02', entonces one_hot_dict['CF02'][0] = 1
for idx, val in datos['Procedencia'].items():
    one_hot_dict_pro[val][idx] = 1

# Convertir el diccionario a un DataFrame
one_hot_dict_pro_df = pd.DataFrame(one_hot_dict_pro)

#Unir el DataFrame original con el codificado
datos = pd.concat([datos, one_hot_dict_pro_df], axis=1)

# # Mostrar el resultado
datos[['Procedencia','TAJ', 'SUL', 'OXI', 'NoDefinido']].head()

#### 3. Hacemos un tratamiento de variable Boleana para convertir a Numerica por el Metodo del Mapping

Variable al_termino_cargar_en_espera_cuadrado_cuadrandose

In [0]:
# # Crear un diccionario para mapear valores booleanos a numéricos
bool_to_num = {True: 1, False: 0}

# # Aplicar el diccionario a la columna booleana para convertirla a numérica
datos['al_termino_cargar_en_espera_cuadrado_cuadrandose'] = datos['al_termino_cargar_en_espera_cuadrado_cuadrandose'].map(bool_to_num)

# Mostrar el resultado
datos['al_termino_cargar_en_espera_cuadrado_cuadrandose'].head()

In [0]:
datos['al_termino_cargar_en_espera_cuadrado_cuadrandose'].value_counts()

### 4. Seleccion de Variables

3.1 Seleccion variables con  StatModels, para regresion lineal (variables significativas estadisticamente basado en la Hipotesis Nula)

In [0]:
#Veamos el tipo de datos que estaremos utilizando
datos[['tonelaje_inicial_poligono_creado','radiohexagonocuchara_equipocarguio',
'capacidad_en_volumen_equipo_carguio_m3','capacidad_en_volumen_equipo_acarreo_m3',
'capacidad_en_peso_equipo_carguio',
'capacidad_en_peso_equipo_acarreo',
'tiempo_estimado_duracion_estado_pala',
'radio_locacion_metros',
#'tiempo_ready_llegada_esperando',
'tonelaje_camion_antes_cargaestabilizada',
'tonelaje_segun_computadora',
'densidad_inicial_poligono_creado_tn/m3','tiempo_carga','al_termino_cargar_en_espera_cuadrado_cuadrandose',
'cantidad_equipos_espera_al_termino_carga_pala','horaini_turnocarga','elevacion_poligono_metros',
'PH003', 'CF002', 'CF001', 'PH001', 'PH002',
'material_8','material_7','material_1','material_4','material_5','material_9','material_6','material_12','material_13','material_11', 'material_2','material_15']].dtypes

In [0]:
datos['capacidad_en_peso_equipo_acarreo'].unique()

In [0]:
datos['numero_pases_carguio'].unique()

In [0]:
import pandas as pd
import statsmodels.api as sm

# Supongamos que datos es tu DataFrame y has seleccionado tus características (X) y variable objetivo (y)
X = datos[['tonelaje_inicial_poligono_creado','radiohexagonocuchara_equipocarguio',
'capacidad_en_volumen_equipo_carguio_m3','capacidad_en_volumen_equipo_acarreo_m3',
'capacidad_en_peso_equipo_carguio',
'capacidad_en_peso_equipo_acarreo',
'tiempo_estimado_duracion_estado_pala',
'radio_locacion_metros',
#'tiempo_ready_llegada_esperando',
'tonelaje_camion_antes_cargaestabilizada',
'tonelaje_segun_computadora',
'densidad_inicial_poligono_creado_tn/m3','tiempo_carga','al_termino_cargar_en_espera_cuadrado_cuadrandose',
'cantidad_equipos_espera_al_termino_carga_pala','horaini_turnocarga','elevacion_poligono_metros',
'PH003', 'CF002', 'CF001', 'PH001', 'PH002',
'material_8','material_7','material_1','material_4','material_5','material_9','material_6','material_12','material_13','material_11', 'material_2','material_15']]  # Reemplaza 'feature1', 'feature2', 'feature3' con tus nombres de características
y = datos['numero_pases_carguio'] # Reemplaza 'variable_objetivo' con el nombre de tu variable objetivo

# Añadir constante a las características (intercepto)
X = sm.add_constant(X)

# Crear el modelo de regresión lineal con statsmodels
modelo = sm.OLS(y, X).fit()

# Imprimir un resumen estadistico del modelo
print(modelo.summary())

3.2 Ver Correlaciones entre mis variables

In [0]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Supongamos que tienes un DataFrame llamado 'datos'

# Lista de variables que deseas analizar
variables_a_analizar = ['tonelaje_inicial_poligono_creado','radiohexagonocuchara_equipocarguio',
'capacidad_en_volumen_equipo_carguio_m3','capacidad_en_volumen_equipo_acarreo_m3',
'capacidad_en_peso_equipo_carguio',
'capacidad_en_peso_equipo_acarreo',
'tiempo_estimado_duracion_estado_pala',
'radio_locacion_metros',
#'tiempo_ready_llegada_esperando',
'tonelaje_camion_antes_cargaestabilizada',
'tonelaje_segun_computadora',
'densidad_inicial_poligono_creado_tn/m3','tiempo_carga','al_termino_cargar_en_espera_cuadrado_cuadrandose',
'cantidad_equipos_espera_al_termino_carga_pala','horaini_turnocarga','elevacion_poligono_metros',
'PH003', 'CF002', 'CF001', 'PH001', 'PH002',
'material_8','material_7','material_1','material_4','material_5','material_9','material_6','material_12','material_13','material_11', 'material_2','material_15','numero_pases_carguio']

# Filtrar el DataFrame para incluir solo las variables de interés
datos_subset = datos[variables_a_analizar]

# Crear una matriz de correlación para el subconjunto de variables
matriz_correlacion = datos_subset.corr()

# Configurar el estilo de Seaborn
sns.set(style="whitegrid")

# Aumentar el tamaño del gráfico para mostrar más detalles
plt.figure(figsize=(20, 8))

# Crear un mapa de calor (heatmap) de la matriz de correlación con desplazamiento habilitado
ax = sns.heatmap(matriz_correlacion, annot=True, fmt=".2f", cmap="coolwarm", linewidths=.5)

# Agregar etiquetas a los ejes X e Y
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, horizontalalignment='right', fontsize=8)
ax.set_yticklabels(ax.get_yticklabels(), rotation=0, horizontalalignment='right', fontsize=8)

# Ajustar el tamaño de las fuentes de los nombres de las variables (eje x)
ax.tick_params(axis='x', labelsize=12)

# Ajustar el tamaño de las fuentes de los nombres de las variables (eje y)
ax.tick_params(axis='y', labelsize=12)

plt.title("Matriz de Correlación Variables")
plt.show()

In [0]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Supongamos que tienes un DataFrame llamado 'datos'

# Lista de variables que deseas analizar
variables_a_analizar = ['tonelaje_inicial_poligono_creado','radiohexagonocuchara_equipocarguio',
'capacidad_en_volumen_equipo_carguio_m3','capacidad_en_volumen_equipo_acarreo_m3',
'capacidad_en_peso_equipo_carguio',
'capacidad_en_peso_equipo_acarreo',
#'tiempo_estimado_duracion_estado_pala',
#'radio_locacion_metros',
#'tiempo_ready_llegada_esperando',
#'tonelaje_camion_antes_cargaestabilizada',
#'tonelaje_segun_computadora',
#'densidad_inicial_poligono_creado_tn/m3',
'tiempo_carga',
#'al_termino_cargar_en_espera_cuadrado_cuadrandose',
#'cantidad_equipos_espera_al_termino_carga_pala','horaini_turnocarga','elevacion_poligono_metros',
'PH003', 'CF002', 'CF001', 'PH001', 'PH002',
#'material_8','material_7','material_1','material_4','material_5','material_9','material_6','material_12','material_13','material_11', 'material_2','material_15',
'numero_pases_carguio']

# Filtrar el DataFrame para incluir solo las variables de interés
datos_subset = datos[variables_a_analizar]

# Crear una matriz de correlación para el subconjunto de variables
matriz_correlacion = datos_subset.corr()

# Configurar el estilo de Seaborn
sns.set(style="whitegrid")

# Aumentar el tamaño del gráfico para mostrar más detalles
plt.figure(figsize=(20, 8))

# Crear un mapa de calor (heatmap) de la matriz de correlación con desplazamiento habilitado
ax = sns.heatmap(matriz_correlacion, annot=True, fmt=".2f", cmap="coolwarm", linewidths=.5)

# Agregar etiquetas a los ejes X e Y
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, horizontalalignment='right', fontsize=8)
ax.set_yticklabels(ax.get_yticklabels(), rotation=0, horizontalalignment='right', fontsize=8)

# Ajustar el tamaño de las fuentes de los nombres de las variables (eje x)
ax.tick_params(axis='x', labelsize=12)

# Ajustar el tamaño de las fuentes de los nombres de las variables (eje y)
ax.tick_params(axis='y', labelsize=12)

plt.title("Matriz de Correlación Variables")
plt.show()

In [0]:
# Variables seleccionadas con 20% de correlacion o superior
# 'radiohexagonocuchara_equipocarguio',
# 'capacidad_en_volumen_equipo_carguio_m3',
# 'capacidad_en_peso_equipo_carguio',
# 'tiempo_estimado_duracion_estado_pala',
# 'CF02','CF01', 'CF03', 'PH03', 'PH02', 'PH01', 'FR01',
# 'TAJ', 'SUL', 'OXI', 'NoDefinido'

3.3 Ver la varianza de mis variables

In [0]:
datos_varianza=datos[['tonelaje_inicial_poligono_creado','radiohexagonocuchara_equipocarguio',
'capacidad_en_volumen_equipo_carguio_m3','capacidad_en_volumen_equipo_acarreo_m3',
'capacidad_en_peso_equipo_carguio',
'capacidad_en_peso_equipo_acarreo',
'tiempo_estimado_duracion_estado_pala',
'radio_locacion_metros',
#'tiempo_ready_llegada_esperando',
'tonelaje_camion_antes_cargaestabilizada',
'tonelaje_segun_computadora',
'densidad_inicial_poligono_creado_tn/m3','tiempo_carga','al_termino_cargar_en_espera_cuadrado_cuadrandose',
'cantidad_equipos_espera_al_termino_carga_pala','horaini_turnocarga','elevacion_poligono_metros',
'PH003', 'CF002', 'CF001', 'PH001', 'PH002',
'material_8','material_7','material_1','material_4','material_5','material_9','material_6','material_12','material_13','material_11', 'material_2','material_15',]]

In [0]:
desviacion_estandar_sorted

In [0]:
#Varibales a descartar porque no aportan informacion al Modelo
#capacidad_en_peso_equipo_acarreo                         0.000000
# capacidad_en_volumen_equipo_acarreo_m3                   0.000000

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Calcular la varianza de cada columna
desviacion_estandar = np.sqrt(datos_varianza.var())  #desviacion estandar 

# Redondear la varianza a 2 decimales
#variance_rounded = variance.round(2)

# Ordenar las varianzas de mayor a menor
desviacion_estandar_sorted = desviacion_estandar.sort_values(ascending=False)

# Graficar los resultados
plt.figure(figsize=(10, 6))
desviacion_estandar_sorted.plot(kind='bar')
plt.title('Varianza de Variables del DataFrame')
plt.xlabel('Variables')
plt.ylabel('Varianza')
# plt.xticks(rotation=45)
plt.show()

#### Variables Seleccionadas antes de entrenar los modelos de ML

In [0]:
datos_subset=datos[['tonelaje_inicial_poligono_creado','radiohexagonocuchara_equipocarguio',
'capacidad_en_volumen_equipo_carguio_m3','capacidad_en_volumen_equipo_acarreo_m3',
'capacidad_en_peso_equipo_carguio',
'capacidad_en_peso_equipo_acarreo',
'tiempo_estimado_duracion_estado_pala',
'radio_locacion_metros',
#'tiempo_ready_llegada_esperando',
'tonelaje_camion_antes_cargaestabilizada',
'tonelaje_segun_computadora',
'densidad_inicial_poligono_creado_tn/m3','tiempo_carga','al_termino_cargar_en_espera_cuadrado_cuadrandose',
'cantidad_equipos_espera_al_termino_carga_pala','horaini_turnocarga','elevacion_poligono_metros',
'PH003', 'CF002', 'CF001', 'PH001', 'PH002',
'material_8','material_7','material_1','material_4','material_5','material_9','material_6','material_12','material_13','material_11', 'material_2','material_15']]
datos_subset.head()

###4. Funcion para Escalar datos, y guardar los valores para escalar en produccion

4.1 Hallamos los Minimos y Maximos de cada Variable a escalar

In [0]:
# Selecciona las variables específicas (no incluirian las varibales boleanas, y las one hot encoding)
variables_especificas = ['tonelaje_inicial_poligono_creado','radiohexagonocuchara_equipocarguio',
'capacidad_en_volumen_equipo_carguio_m3','capacidad_en_volumen_equipo_acarreo_m3',
'capacidad_en_peso_equipo_carguio',
#'capacidad_en_peso_equipo_acarreo',
'tiempo_estimado_duracion_estado_pala',
'radio_locacion_metros',
#'tiempo_ready_llegada_esperando',
'tonelaje_camion_antes_cargaestabilizada',
'tonelaje_segun_computadora',
'densidad_inicial_poligono_creado_tn/m3','tiempo_carga','al_termino_cargar_en_espera_cuadrado_cuadrandose',
'cantidad_equipos_espera_al_termino_carga_pala','horaini_turnocarga','elevacion_poligono_metros',
#'PH003', 'CF002', 'CF001', 'PH001', 'PH002',
#'material_8','material_7','material_1','material_4','material_5','material_9','material_6','material_12','material_13','material_11', 'material_2',
# 'material_15'
]

# Crear un DataFrame con las variables específicas
datos_subset_escalar = datos_subset[variables_especificas]

In [0]:
X_min = datos_subset_escalar.min()
X_max = datos_subset_escalar.max()
X_min, X_max

4.2 Generamos la funcion para generar el escalamiento que dejara valores entre 0 y 1 

In [0]:
X_min.values

In [0]:
# X_minimo = np.array([-242063.328,
#                      12.000, 
#                      0.000,
#                      464.000,
#                     0.000,
#                      -1.000 ])
# X_maximo = np.array([3.142382e+03,
#                      2.700000e+01, 
#    4.241000e+00, 
#    8.370000e+02,   
#    2.000000e+07,
#    8.000000e+00])

In [0]:
X_minimo = np.array([  0.000e+00,  0.000e+00,  1.900e+01,  1.240e+02,  3.500e+01,
        0.000e+00,  5.000e+00,  0.000e+00,  1.501e+02,  0.000e+00,
        0.000e+00,  1.000e+00, -1.000e+00,  7.000e+00,  3.990e+03])
X_maximo = np.array([3.90767029e+06, 3.50000000e+02, 2.70000000e+01, 1.44000000e+02,
       4.90000000e+01, 0.00000000e+00, 8.00000000e+01, 3.64600000e+03,
       3.03800000e+02, 2.25000000e+01, 2.87058300e+03, 1.00000000e+00,
       6.00000000e+00, 1.90000000e+01, 4.36500000e+03])
def min_max_scale(data, X_min, X_max):
    '''
    Parámetros:
    - data: Un array NumPy con los datos crudos.
    - X_min: Un array NumPy con los valores mínimos para cada característica.
    - X_max: Un array NumPy con los valores máximos para cada característica.
    '''
    data_scaled = (data - X_min) / (X_max - X_min)
    return data_scaled
datos_subset_escalados = min_max_scale(datos_subset_escalar,X_minimo, X_maximo)

datos_subset_escalados = datos_subset_escalados.fillna(0) 

datos_subset_escalados.head()

In [0]:
datos_subset_escalados = datos_subset_escalados.fillna(0) 

In [0]:
# 3 Verificamos que no haya valores NULL luego de hacer el Escalamiento de Datos
datos_subset_escalados.isnull().sum()

In [0]:
# Reemplazar las columnas escaladas en el DataFrame original
datos_subset[variables_especificas] = datos_subset_escalados

# Mostrar el resultado
datos_subset.head()

In [0]:
#Finalmente tenemos nuestros datos escalados para variables especificos (incluidos las variables que no han sido escaldas)
datos_total_escalados = datos_subset

4.3 Probando como escalariamos en produccion, real time

In [0]:
# Supongamos que esta es la fila de entrada con los valores crudos
X_crudo = np.array([15, 7, 2.0, 45, 15])
X_crudo

In [0]:
min = np.array([12.,  0.,  0.,  0.,  0.])
max = np.array([2.700000e+01, 4.241000e+00, 5.012661e+03, 2.000000e+07,
       4.000000e+02])
X_crudo = np.array([15, 7, 2.0, 45, 15])
min_max_scale(X_crudo,min,max)

### 5. Implementacion de Modelos de Machine learning y Deep Learning

#### 5.1 Seleccionado tus características (X) y variable objetivo (y)

In [0]:
datos_total_escalados.columns

In [0]:
datos_total_escalados.head()

#### Luego de Hacer el Feature Importances (Metodo de Envoltura) se identifico este conjunto de variables mas importantes

In [0]:
### Luego de Hacer el Feature Importances (Metodo de Envoltura) se identifico este conjunto de variables mas importantes
datos_total_escalados_variablesML =  datos_total_escalados[['tonelaje_inicial_poligono_creado',
       #'radiohexagonocuchara_equipocarguio',
       'capacidad_en_volumen_equipo_carguio_m3',
       #'capacidad_en_volumen_equipo_acarreo_m3',
       'capacidad_en_peso_equipo_carguio', 'capacidad_en_peso_equipo_acarreo',
       #'tiempo_estimado_duracion_estado_pala', 'radio_locacion_metros',
       #'tonelaje_camion_antes_cargaestabilizada', #'tonelaje_segun_computadora',
       'densidad_inicial_poligono_creado_tn/m3', 'tiempo_carga',
       #'al_termino_cargar_en_espera_cuadrado_cuadrandose',
       #'cantidad_equipos_espera_al_termino_carga_pala', #'horaini_turnocarga',
       #'elevacion_poligono_metros',
        'PH003', 'CF002', 'CF001', 'PH001',
       'PH002', 
       #'material_8', 'material_7', 'material_1', 'material_4',
       #'material_5', 'material_9', 'material_6', 'material_12', #'material_13',
       #'material_11', 'material_2', 'material_15'
       ]]

# Supongamos que datos es tu DataFrame y has seleccionado tus características (X) y variable objetivo (y)
# Capacidad_peso = capacidad_volumen * densidad   O  capacidad_volumen = Capacidad_peso / densidad
X = datos_total_escalados_variablesML.values #Sale de los datos escalados

y = datos['numero_pases_carguio'].values # Reemplaza 'variable_objetivo' con el nombre de tu variable objetivo(sale de los datos originales)

In [0]:
X.shape

#### 5.2 Regresion Lineal

In [0]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics

# Dividir el conjunto de datos en entrenamiento y prueba
X_train_rl, X_test_rl, y_train_rl, y_test_rl = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear el modelo de regresión lineal múltiple
modelo_rl = LinearRegression()

# Entrenar el modelo
modelo_rl.fit(X_train_rl, y_train_rl)

# Realizar predicciones en el conjunto de entrenamiento y prueba
y_pred_train_rl = modelo_rl.predict(X_train_rl)
y_pred_test_rl = modelo_rl.predict(X_test_rl)

# Redondear los valores de y_pred al entero más cercano
y_pred_train_rl = np.round(y_pred_train_rl).astype('int64')
y_pred_test_rl = np.round(y_pred_test_rl).astype('int64')

# Medir el rendimiento del modelo en el conjunto de entrenamiento
# mae_train = metrics.mean_absolute_error(y_train_rl, y_pred_train_rl)
mse_train = metrics.mean_squared_error(y_train_rl, y_pred_train_rl)
rmse_train = np.sqrt(mse_train)
# r2_train = metrics.r2_score(y_train_rl, y_pred_train_rl)

# Medir el rendimiento del modelo en el conjunto de prueba
# mae_test = metrics.mean_absolute_error(y_test_rl, y_pred_test_rl)
mse_test = metrics.mean_squared_error(y_test_rl, y_pred_test_rl)
rmse_test = np.sqrt(mse_test)
# r2_test = metrics.r2_score(y_test_rl, y_pred_test_rl)

# Imprimir métricas de rendimiento en ambos conjuntos
print("Métricas de rendimiento en el conjunto de entrenamiento:")
# print(f'Error Absoluto Medio (MAE): {mae_train}')
# print(f'Error Cuadrático Medio (MSE): {mse_train}')
print(f'Raíz del Error Cuadrático Medio (RMSE): {rmse_train}')
# print(f'Coeficiente de Determinación (R^2): {r2_train}')
print("----------------------------------------------")
print("Métricas de rendimiento en el conjunto de prueba:")
# print(f'Error Absoluto Medio (MAE): {mae_test}')
# print(f'Error Cuadrático Medio (MSE): {mse_test}')
print(f'Raíz del Error Cuadrático Medio (RMSE): {rmse_test}')
# print(f'Coeficiente de Determinación (R^2): {r2_test}')

Un RMSE de 1.22 significa que, en promedio, las predicciones tienen un error de alrededor de 1.52 unidades  de nuemro de pases.

Crear un gráfico de valores reales vs predichos

In [0]:
# Visualizar las predicciones
plt.scatter(y_test_rl, y_pred_test_rl)
plt.xlabel('Real [Pases]')
plt.ylabel('Prediccion [Pases]')
lims = [0, 20]
plt.xlim(lims)
plt.ylim(lims)
plt.plot(lims, lims)

#### 5.3 Arbol de decision Regressor

In [0]:
### Luego de Hacer el Feature Importances (Metodo de Envoltura) se identifico este conjunto de variables mas importantes
datos_total_escalados_dt =  datos_total_escalados[['tonelaje_inicial_poligono_creado',
       #'radiohexagonocuchara_equipocarguio',
       #'capacidad_en_volumen_equipo_carguio_m3',
       #'capacidad_en_volumen_equipo_acarreo_m3',
       'capacidad_en_peso_equipo_carguio', 
       'capacidad_en_peso_equipo_acarreo',
       #'tiempo_estimado_duracion_estado_pala', 'radio_locacion_metros',
       #'tonelaje_camion_antes_cargaestabilizada', 'tonelaje_segun_computadora',
        'densidad_inicial_poligono_creado_tn/m3', 
        'tiempo_carga',
       #'al_termino_cargar_en_espera_cuadrado_cuadrandose',
       #'cantidad_equipos_espera_al_termino_carga_pala', 'horaini_turnocarga',
       #'elevacion_poligono_metros', 
       'PH003', 'CF002', 'CF001', 'PH001','PH002', 
       #'material_8', 'material_7', 'material_1', 'material_4',
       #'material_5', 'material_9', 'material_6', 'material_12', 'material_13',
       #'material_11', 'material_2', 'material_15'
       ]]

# Supongamos que datos es tu DataFrame y has seleccionado tus características (X) y variable objetivo (y)
# Capacidad_peso = capacidad_volumen * densidad   O  capacidad_volumen = Capacidad_peso / densidad
X = datos_total_escalados_dt.values #Sale de los datos escalados

y = datos['numero_pases_carguio'].values # Reemplaza 'variable_objetivo' con el nombre de tu variable objetivo(sale de los datos originales)

In [0]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

# Supongamos que X es tu conjunto de características y y es tu variable objetivo (Número de Pases)
# Asegúrate de tener definidas estas variables antes de ejecutar el código

# Dividir el conjunto de datos en entrenamiento y prueba
X_train_dt, X_test_dt, y_train_dt, y_test_dt = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear el modelo de árbol de decisión para regresión
model_arbol = DecisionTreeRegressor(max_depth=7,random_state=42)

# Entrenar el modelo
model_arbol.fit(X_train_dt, y_train_dt)

#----------------------------------------------------------------------------------------------------------------------
# Realizar predicciones en el conjunto de train
y_pred_dt_train = model_arbol.predict(X_train_dt)

# Redondear los valores de y_pred al entero más cercano
y_pred_dt_train = np.round(y_pred_dt_train).astype('int64')

#------------------------------------------------------------------------------------------------------------------------
# Realizar predicciones en el conjunto de prueba
y_pred_dt_test = model_arbol.predict(X_test_dt)

# Redondear los valores de y_pred al entero más cercano
y_pred_dt_test = np.round(y_pred_dt_test).astype('int64')

#-------------------------------------------------------------------------------------------------------------------------

# Visualizar el árbol de decisión (opcional)
# Nota: Necesitarás tener instalada la biblioteca graphviz y pydotplus para esto
# from sklearn.tree import export_graphviz
# import pydotplus
# dot_data = export_graphviz(modelo_arbol, out_file=None, feature_names=X.columns, filled=True, rounded=True)
# graph = pydotplus.graph_from_dot_data(dot_data)
# graph.write_png('arbol_decision.png')

# Medir el rendimiento del modelo
# error_absoluto_medio_dt = metrics.mean_absolute_error(y_test_dt, y_pred_dt)
# error_cuadratico_medio_dt = metrics.mean_squared_error(y_test_dt, y_pred_dt)
raiz_error_cuadratico_medio_dt_train = metrics.mean_squared_error(y_train_dt, y_pred_dt_train, squared=False) #squared=False indica a la #función que calcule la raíz cuadrada 
raiz_error_cuadratico_medio_dt_test = metrics.mean_squared_error(y_test_dt, y_pred_dt_test, squared=False)

# Imprimir métricas de rendimiento
# print(f'Error Absoluto Medio: {error_absoluto_medio_dt}')
# print(f'Error Cuadrático Medio: {error_cuadratico_medio_dt}')
print(f'Raíz del Error Cuadrático Medio Train: {raiz_error_cuadratico_medio_dt_train}')
print(f'Raíz del Error Cuadrático Medio Test: {raiz_error_cuadratico_medio_dt_test}')

Extraer mediante el metodo de envoltura las "Variables mas importantes para el modelo"

In [0]:
# Obtener las importancias de las características
feature_importances_dt = model_arbol.feature_importances_

# Crear un DataFrame para mostrar las importancias de las características junto con sus nombres
feature_importance_df_dt = pd.DataFrame({
    'Feature': datos_total_escalados_dt.columns,
    'Importance': feature_importances_dt
})

# Ordenar el DataFrame por importancia en orden descendente
feature_importance_df_dt = feature_importance_df_dt.sort_values(by='Importance', ascending=False)

# Mostrar las 10 características más importantes
print(feature_importance_df_dt.head(40))

In [0]:
# Obtener las importancias de las características
feature_importances_dt = model_arbol.feature_importances_

# Crear un DataFrame para mostrar las importancias de las características junto con sus nombres
feature_importance_df_dt = pd.DataFrame({
    'Feature': datos_total_escalados.columns,
    'Importance': feature_importances_dt
})

# Ordenar el DataFrame por importancia en orden descendente
feature_importance_df_dt = feature_importance_df_dt.sort_values(by='Importance', ascending=False)

# Mostrar las 10 características más importantes
print(feature_importance_df_dt.head(40))

#### 5.4 Random Forest 

In [0]:
### Luego de Hacer el Feature Importances (Metodo de Envoltura) se identifico este conjunto de variables mas importantes
datos_total_escalados_rf =  datos_total_escalados[[
        #'tonelaje_inicial_poligono_creado',
       #'radiohexagonocuchara_equipocarguio',
       #'capacidad_en_volumen_equipo_carguio_m3',
       #'capacidad_en_volumen_equipo_acarreo_m3',
       'capacidad_en_peso_equipo_carguio', 
       'capacidad_en_peso_equipo_acarreo',
       #'tiempo_estimado_duracion_estado_pala', 'radio_locacion_metros',
       #'tonelaje_camion_antes_cargaestabilizada', 'tonelaje_segun_computadora',
        'densidad_inicial_poligono_creado_tn/m3', 
        'tiempo_carga',
       #'al_termino_cargar_en_espera_cuadrado_cuadrandose',
       'cantidad_equipos_espera_al_termino_carga_pala', 'horaini_turnocarga',
       'elevacion_poligono_metros', 
       'PH003', 'CF002', 'CF001', 'PH001','PH002', 
       #'material_8', 'material_7', 'material_1', 'material_4',
       #'material_5', 'material_9', 'material_6', 'material_12', #'material_13',
       #'material_11', 'material_2', 'material_15'
       ]]

# Supongamos que datos es tu DataFrame y has seleccionado tus características (X) y variable objetivo (y)
# Capacidad_peso = capacidad_volumen * densidad   O  capacidad_volumen = Capacidad_peso / densidad
X = datos_total_escalados_rf.values #Sale de los datos escalados

y = datos['numero_pases_carguio'].values # Reemplaza 'variable_objetivo' con el nombre de tu variable objetivo(sale de los datos originales)

In [0]:
# Importar librerías necesarias
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt

# Supongamos que tienes un DataFrame llamado df con tus datos, y la columna objetivo es 'target'
# Asegúrate de tener las características adecuadas para entrenar el modelo

# Dividir el conjunto de datos en entrenamiento y prueba
# Dividir el conjunto de datos en entrenamiento y prueba
X_train_rf, X_test_rf, y_train_rf, y_test_rf = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear el modelo de Random Forest
model_rf = RandomForestRegressor(n_estimators=50, max_depth=15 ,random_state=42)

# Entrenar el modelo
model_rf.fit(X_train_rf, y_train_rf)

#----------------------------------------------------------------------------------------------------------------------
# Realizar predicciones en el conjunto de train
y_pred_rf_train = model_rf.predict(X_train_rf)

# Redondear los valores de y_pred al entero más cercano
y_pred_rf_train = np.round(y_pred_rf_train).astype('int64')

#------------------------------------------------------------------------------------------------------------------------
# Realizar predicciones en el conjunto de prueba
y_pred_rf_test = model_rf.predict(X_test_rf)

# Redondear los valores de y_pred al entero más cercano
y_pred_rf_test = np.round(y_pred_rf_test).astype('int64')

#-------------------------------------------------------------------------------------------------------------------------

# Medir el rendimiento del modelo
# error_absoluto_medio_dt = metrics.mean_absolute_error(y_test_dt, y_pred_dt)
# error_cuadratico_medio_dt = metrics.mean_squared_error(y_test_dt, y_pred_dt)
raiz_error_cuadratico_medio_rf_train = metrics.mean_squared_error(y_train_rf, y_pred_rf_train, squared=False) #squared=False indica a la #función que calcule la raíz cuadrada 
raiz_error_cuadratico_medio_rf_test = metrics.mean_squared_error(y_test_rf, y_pred_rf_test, squared=False)

# Imprimir métricas de rendimiento
# print(f'Error Absoluto Medio: {error_absoluto_medio_dt}')
# print(f'Error Cuadrático Medio: {error_cuadratico_medio_dt}')
print(f'Raíz del Error Cuadrático Medio Train: {raiz_error_cuadratico_medio_rf_train}')
print(f'Raíz del Error Cuadrático Medio Test: {raiz_error_cuadratico_medio_rf_test}')

Extraer mediante el metodo de envoltura las "Variables mas importantes para el modelo"

In [0]:
# Obtener las importancias de las características
feature_importances_rf = model_rf.feature_importances_

# Crear un DataFrame para mostrar las importancias de las características junto con sus nombres
feature_importance_df_rf = pd.DataFrame({
    'Feature': datos_total_escalados_rf.columns,
    'Importance': feature_importances_rf
})

# Ordenar el DataFrame por importancia en orden descendente
feature_importance_df_rf = feature_importance_df_rf.sort_values(by='Importance', ascending=False)

# Mostrar las 10 características más importantes
print(feature_importance_df_rf.head(40))

In [0]:
# Obtener las importancias de las características
feature_importances_rf = model_rf.feature_importances_

# Crear un DataFrame para mostrar las importancias de las características junto con sus nombres
feature_importance_df_rf = pd.DataFrame({
    'Feature': datos_total_escalados.columns,
    'Importance': feature_importances_rf
})

# Ordenar el DataFrame por importancia en orden descendente
feature_importance_df_rf = feature_importance_df_rf.sort_values(by='Importance', ascending=False)

# Mostrar las 10 características más importantes
print(feature_importance_df_rf.head(40))

#### 5.5 SVM (Support Vector Machine)

In [0]:
from sklearn.svm import SVR

# Dividir el conjunto de datos en entrenamiento y prueba
X_train_svm, X_test_svm, y_train_svm, y_test_svm = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear y entrenar el modelo de SVM Regressor
model_svm = SVR(kernel='linear')  # Puedes ajustar los hiperparámetros según sea necesario
model_svm.fit(X_train_svm, y_train_svm)

# Realizar predicciones en el conjunto de prueba con SVM Regressor
y_pred_svm = model_svm.predict(X_test_svm)
y_pred_svm = np.round(y_pred_svm).astype('int64')

# Medir el rendimiento del modelo de SVM Regressor
error_absoluto_medio_svm = metrics.mean_absolute_error(y_test_svm, y_pred_svm)
error_cuadratico_medio_svm = metrics.mean_squared_error(y_test_svm, y_pred_svm)
raiz_error_cuadratico_medio_svm = metrics.mean_squared_error(y_test_svm, y_pred_svm, squared=False)

# Imprimir métricas de rendimiento de SVM Regressor
print("\nMétricas de rendimiento para SVM Regressor:")
print(f'Error Absoluto Medio: {error_absoluto_medio_svm}')
print(f'Error Cuadrático Medio: {error_cuadratico_medio_svm}')
print(f'Raíz del Error Cuadrático Medio: {raiz_error_cuadratico_medio_svm}')

#### 5.6 XGBoost: 

Construye árboles de manera secuencial. Cada árbol se entrena para corregir los errores de los árboles anteriores.

In [0]:
### Luego de Hacer el Feature Importances (Metodo de Envoltura) se identifico este conjunto de variables mas importantes
datos_total_escalados_xgb =  datos_total_escalados[[
    'tonelaje_inicial_poligono_creado',
       #'radiohexagonocuchara_equipocarguio',
       'capacidad_en_volumen_equipo_carguio_m3',
       #'capacidad_en_volumen_equipo_acarreo_m3',
       #'capacidad_en_peso_equipo_carguio', 
       #'capacidad_en_peso_equipo_acarreo',
       #'tiempo_estimado_duracion_estado_pala', 
       'radio_locacion_metros',
       #'tonelaje_camion_antes_cargaestabilizada', #'tonelaje_segun_computadora',
        #'densidad_inicial_poligono_creado_tn/m3', 
        'tiempo_carga',
       #'al_termino_cargar_en_espera_cuadrado_cuadrandose',
       'cantidad_equipos_espera_al_termino_carga_pala', 'horaini_turnocarga',
       'elevacion_poligono_metros', 
       'PH003', 'CF002', 'CF001', 'PH001','PH002', 
       'material_8', 'material_7', 'material_1', 'material_4',
       'material_5', 'material_9', 'material_6', 'material_12', 'material_13',
       'material_11', 'material_2', 'material_15',
       'capacidad_en_peso_equipo_acarreo'
       ]]

# Supongamos que datos es tu DataFrame y has seleccionado tus características (X) y variable objetivo (y)
# Capacidad_peso = capacidad_volumen * densidad   O  capacidad_volumen = Capacidad_peso / densidad
X = datos_total_escalados_xgb.values #Sale de los datos escalados

y = datos['numero_pases_carguio'].values # Reemplaza 'variable_objetivo' con el nombre de tu variable objetivo(sale de los datos originales)

#### Metodo 1,  Sin convertir a ONNX

In [0]:
# Instalar las bibliotecas necesarias
# pip install xgboost scikit-learn matplotlib
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import xgboost as xgb
import matplotlib.pyplot as plt

# Supongamos que tienes un DataFrame llamado df con tus datos, y la columna objetivo es 'target'
# Asegúrate de tener las características adecuadas para entrenar el modelo

# Dividir el conjunto de datos en entrenamiento y prueba
X_train_xgb, X_test_xgb, y_train_xgb, y_test_xgb = train_test_split(X, y, test_size=0.1, random_state=42)

# Crear el modelo de XGBoost
model_xgb = xgb.XGBRegressor(
    n_estimators=300,  # Número de árboles
    max_depth=15,  # Profundidad máxima de los árboles
    learning_rate=0.03, # tasa de aprendizaje, mas lento en ajustarse, pero necesitarias mas arboles
    subsample = 0.7,
    #reg_alpha = 0.5,
    reg_lambda = 2.5, # regulazirzacion L2 Ridge
    random_state=42 #Fijar la aleatoriedad durante el entrenamiento 
)

# Entrenar el modelo
model_xgb.fit(X_train_xgb, y_train_xgb)

#----------------------------------------------------------------------------------------------------------------------
# Realizar predicciones en el conjunto de train
y_pred_xgb_train = model_xgb.predict(X_train_xgb)

# Redondear los valores de y_pred al entero más cercano
y_pred_xgb_train = np.round(y_pred_xgb_train).astype('int64')

#------------------------------------------------------------------------------------------------------------------------
# Realizar predicciones en el conjunto de prueba
y_pred_xgb_test = model_xgb.predict(X_test_xgb)

# Redondear los valores de y_pred al entero más cercano
y_pred_xgb_test = np.round(y_pred_xgb_test).astype('int64')

#-------------------------------------------------------------------------------------------------------------------------

# Medir el rendimiento del modelo
# error_absoluto_medio_dt = metrics.mean_absolute_error(y_test_dt, y_pred_dt)
# error_cuadratico_medio_dt = metrics.mean_squared_error(y_test_dt, y_pred_dt)
raiz_error_cuadratico_medio_xgb_train = metrics.mean_squared_error(y_train_xgb, y_pred_xgb_train, squared=False) #squared=False indica a la #función que calcule la raíz cuadrada 
raiz_error_cuadratico_medio_xgb_test = metrics.mean_squared_error(y_test_xgb, y_pred_xgb_test, squared=False)

# Imprimir métricas de rendimiento
# print(f'Error Absoluto Medio: {error_absoluto_medio_dt}')
# print(f'Error Cuadrático Medio: {error_cuadratico_medio_dt}')
print(f'Raíz del Error Cuadrático Medio Train: {raiz_error_cuadratico_medio_xgb_train}')
print(f'Raíz del Error Cuadrático Medio Test: {raiz_error_cuadratico_medio_xgb_test}')

#### Metodo 2,  Con convertir a ONNX

In [0]:
import numpy as np  #Libreria para manejo de operaciones matematicas y matrices
#pip install onnxruntime -- Instalamos esta libreria
import onnxruntime as rt   # operaciones con onnxruntime, para cargar el modelo.onnx
#onnxruntime es una librería de ejecución de modelos ONNX (Open Neural Network Exchange) que permite cargar y
# ejecutar modelos ONNX en diferentes plataformas y dispositivos.
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
# from xgboost import XGBClassifier, XGBRegressor, DMatrix, train as train_xgb

#pip install --user skl2onnx -- Instala esta libreria como usuario, para no tener problemas de acceso
from skl2onnx.common.data_types import FloatTensorType
from skl2onnx import convert_sklearn, to_onnx, update_registered_converter
from skl2onnx.common.shape_calculator import (
    calculate_linear_classifier_output_shapes,
    calculate_linear_regressor_output_shapes,
)
from onnxmltools.convert import convert_xgboost as convert_xgboost_booster
from onnxmltools.convert.xgboost.operator_converters.XGBoost import convert_xgboost
#skl2onnx es una librería que proporciona herramientas para convertir modelos sklearn a 
#formato ONNX. onnxmltools es otra librería que proporciona herramientas para convertir modelos xgboost a formato ONNX. 
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import xgboost as xgb
import matplotlib.pyplot as plt

# Registro del convertidor XGB CLASIFICADOR
# update_registered_converter(
#     XGBClassifier,
#     "XGBoostXGBClassifier",
#     calculate_linear_classifier_output_shapes,
#     convert_xgboost,
#     options={"nocl": [True, False], "zipmap": [True, False, "columns"]},
# )

# Registro del convertidor XGB REGRESSOR
update_registered_converter(
    xgb.XGBRegressor,
    "XGBoostXGBRegressor",
    calculate_linear_regressor_output_shapes,
    convert_xgboost
)

# Supongamos que tienes un DataFrame llamado df con tus datos, y la columna objetivo es 'target'
# Asegúrate de tener las características adecuadas para entrenar el modelo

# Dividir el conjunto de datos en entrenamiento y prueba
X_train_xgb, X_test_xgb, y_train_xgb, y_test_xgb = train_test_split(X, y, test_size=0.1, random_state=42)

# Crear el modelo de XGBoost
model_pipe_xgb = Pipeline([("xgb", xgb.XGBRegressor(
    n_estimators=300,  # Número de árboles
    max_depth=15,  # Profundidad máxima de los árboles
    learning_rate=0.03, # tasa de aprendizaje, mas lento en ajustarse, pero necesitarias mas arboles
    subsample = 0.7,# Proporción de entrenamiento utilizadas cada árbol
    #reg_alpha = 0.5, # regularizacion L1 Lasso (restringe a 0)
    reg_lambda = 2.5, # regulazirzacion L2 Ridge (restringe al minimo)
    random_state=42 #Fijar la aleatoriedad durante el entrenamiento 
))])


# Entrenar el modelo
model_pipe_xgb.fit(X_train_xgb, y_train_xgb)

#----------------------------------------------------------------------------------------------------------------------
# Realizar predicciones en el conjunto de train
y_pred_xgb_train = model_pipe_xgb.predict(X_train_xgb)

# Redondear los valores de y_pred al entero más cercano
y_pred_xgb_train = np.round(y_pred_xgb_train).astype('int64')

#------------------------------------------------------------------------------------------------------------------------
# Realizar predicciones en el conjunto de prueba
y_pred_xgb_test = model_pipe_xgb.predict(X_test_xgb)

# Redondear los valores de y_pred al entero más cercano
y_pred_xgb_test = np.round(y_pred_xgb_test).astype('int64')

Medir el rendimiento del modelo

In [0]:
# Medir el rendimiento del modelo
# error_absoluto_medio_dt = metrics.mean_absolute_error(y_test_dt, y_pred_dt)
# error_cuadratico_medio_dt = metrics.mean_squared_error(y_test_dt, y_pred_dt)
raiz_error_cuadratico_medio_xgb_train = metrics.mean_squared_error(y_train_xgb, y_pred_xgb_train, squared=False) #squared=False indica a la #función que calcule la raíz cuadrada 
raiz_error_cuadratico_medio_xgb_test = metrics.mean_squared_error(y_test_xgb, y_pred_xgb_test, squared=False)

# Imprimir métricas de rendimiento
# print(f'Error Absoluto Medio: {error_absoluto_medio_dt}')
# print(f'Error Cuadrático Medio: {error_cuadratico_medio_dt}')
print(f'Raíz del Error Cuadrático Medio Train: {raiz_error_cuadratico_medio_xgb_train}')
print(f'Raíz del Error Cuadrático Medio Test: {raiz_error_cuadratico_medio_xgb_test}')

#### Convertir el Modelo Pipe a ONNX

In [0]:
model_onnx = convert_sklearn(
    model_pipe_xgb,
    "xgboost_v1",
    [("input", FloatTensorType([None, X.shape[1]]))],
    target_opset={"": 12, "ai.onnx.ml": 2},
)

# And save.
with open("xgboost_v1.onnx", "wb") as f:
    f.write(model_onnx.SerializeToString())

Hacer una prediccion (Inferencia) con el Modelo ONNX convertido

In [0]:
X[:2]

In [0]:
sess = rt.InferenceSession("xgboost_v1.onnx", providers=["CPUExecutionProvider"])
pred_onx = sess.run(None, {"input": X[:2].astype(np.float32)})
print("predict", pred_onx[0])

In [0]:
# from onnxmltools.convert import convert_xgboost as convert_xgboost_booster
# import onnxmltools

# # Convertir el modelo XGBoost a ONNX
# onnx_model = convert_xgboost_booster(model_xgb, 
#                                      'XGBoost model', 
#                                      initial_types=[('input',
#                                                       FloatTensorType([None, X.shape[1]]))])

# # Guardar el modelo ONNX en un archivo
# onnx_file = "xgboost_model.onnx"
# onnxmltools.utils.save_model(onnx_model, onnx_file)

Extraer mediante el metodo de envoltura las "Variables mas importantes para el modelo"

In [0]:
# Obtener las importancias de las características
feature_importances_xgb = model_xgb.feature_importances_

# Crear un DataFrame para mostrar las importancias de las características junto con sus nombres
feature_importance_df_xgb = pd.DataFrame({
    'Feature': datos_total_escalados_xgb.columns,
    'Importance': feature_importances_xgb
})

# Ordenar el DataFrame por importancia en orden descendente
feature_importance_df_xgb = feature_importance_df_xgb.sort_values(by='Importance', ascending=False)

# Mostrar las 10 características más importantes
print(feature_importance_df_xgb.head(40))

In [0]:
# Obtener las importancias de las características
feature_importances_xgb = model_xgb.feature_importances_

# Crear un DataFrame para mostrar las importancias de las características junto con sus nombres
feature_importance_df_xgb = pd.DataFrame({
    'Feature': datos_total_escalados.columns,
    'Importance': feature_importances_xgb
})

# Ordenar el DataFrame por importancia en orden descendente
feature_importance_df_xgb = feature_importance_df_xgb.sort_values(by='Importance', ascending=False)

# Mostrar las 10 características más importantes
print(feature_importance_df_xgb.head(40))

#### Tunnig de Hiperparametros del Modelo con GridSearch

In [0]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV, train_test_split

X_train_xgb, X_test_xgb, y_train_xgb, y_test_xgb = train_test_split(X, y, test_size=0.1, random_state=42)

# Definir los parámetros para la búsqueda
param_grid = {
    'n_estimators': [100, 200, 300],  #Numero  de arboles que entrenaran
    'max_depth': [10, 15, 20],   #profundidad de cada arbol 
    'learning_rate': [0.01], #tasa de aprendizaje, valor más bajo significa que el modelo se ajusta más lentamente,que puede resultar un mejor rendimiento pero requiere un número mayor de árboles
    #'subsample': [0.7],  #Proporción de las muestras de entrenamiento utilizadas para construir cada árbol. Valores menores a 1.0 pueden prevenir el sobreajuste al introducir aleatoriedad  
    #'reg_alpha': [0, 0.1, 0.5]  #Regularización L1 (alpha o Lasso): Útil cuando se desea un modelo más simple y se sospecha que muchas características no son relevantes. ()
    #L1 Penaliza las características menos importantes y puede llevar algunas de ellas a tener un peso cero, esencialmente eliminándolas del modelo
    
    'reg_lambda': [1.5, 2],  # Regularización L2 (lambda o Ridge): Útil cuando se desea un modelo que sea robusto y evite el sobreajuste, pero sin eliminar completamente ninguna característica.
    # Penaliza las características con grandes coeficientes y reduce el impacto de las características menos importantes, sin eliminarlas por completo
}

model_xgb = xgb.XGBRegressor(random_state=42)

# Realizar la búsqueda en la cuadrícula con validación cruzada
grid_search = GridSearchCV(estimator=model_xgb, param_grid=param_grid, cv=2, n_jobs=-1, verbose=2)
#n_jobs=-1, se utilizan todos los núcleos de CPU disponibles en la máquina. Esto permite maximizar el uso de recursos del sistema para acelerar el proceso.
# validación cruzada (CV) divide el conjunto de datos en un número de particiones (folds), entrena el modelo en todas las particiones menos una y valida en la partición restante. Este proceso se repite para cada partición, asegurando que cada parte de los datos se utilice para validación exactamente una vez


grid_search.fit(X_train_xgb, y_train_xgb)

# Imprimir los mejores parámetros y el mejor score
print("Best parameters found: ", grid_search.best_params_)
print("Best score found: ", grid_search.best_score_)

# Evaluar el modelo en el conjunto de prueba
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test_xgb)

In [0]:
# !pip list

#### 5.7 Red Neuronal MLP

In [0]:
# Aseguramos que los resultados sean "reproducibles" en cada ejecucion de tensorflow(pesos iniciales aleatorios)
import tensorflow as tf
import numpy as np

np.random.seed(42)
tf.random.set_seed(42)

In [0]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
import numpy as np
import matplotlib.pyplot as plt

# Suponiendo que tienes tus datos X e y definidos antes de esta sección


#Dividir los sets de entrenamiento, validacion, test
train_ratio = 0.75
validation_ratio = 0.15
test_ratio = 0.10

# train is now 75% of the entire data set (Dejas el 0.75 para entrenamiento 1-0.75 = 0.25 quedan libres)
x_train_mlp, x_test_mlp, y_train_mlp, y_test_mlp = train_test_split(X, y, test_size=1 - train_ratio)

# test is now 10% of the initial data set
# validation is now 15% of the initial data set(de lo que sobro arriba 25%, sacas tu validacion y test)
x_val_mlp, x_test_mlp, y_val_mlp, y_test_mlp = train_test_split(x_test_mlp, y_test_mlp, test_size=test_ratio/(test_ratio + validation_ratio)) 

# ---  test_ratio/(test_ratio + validation_ratio) ---
#Esto significa que del 25% de datos restantes, el 40% será para el conjunto de prueba y el 60% para el conjunto de validación.
#Con esta proporción:
#x_val_lstm y y_val_lstm contendrán 0.60 * 0.25 = 0.15 del total original.
#x_test_lstm y y_test_lstm contendrán 0.40 * 0.25 = 0.10 del total original.


# Crear el modelo de la red neuronal
model_mlp = Sequential()
model_mlp.add(Dense(10, input_dim=X.shape[1], activation='relu')) #input_dim : numero de variables inputs
model_mlp.add(Dense(20, activation='relu'))
model_mlp.add(Dense(10, activation='relu'))
model_mlp.add(Dense(1, activation='linear'))

# Compilar el modelo
model_mlp.compile(loss='mean_squared_error', optimizer='adam')
model_mlp.summary()

# Entrenar el modelo y registrar las pérdidas
# Entrenar el modelo y registrar las pérdidas
history = model_mlp.fit(x_train_mlp, y_train_mlp, epochs=50, batch_size=5, verbose=2, validation_data=(x_val_mlp, y_val_mlp))

# Hacer predicciones en el conjunto de prueba
y_pred_mlp = model_mlp.predict(x_test_mlp)
y_pred_mlp = np.round(y_pred_mlp).astype('int64')

# Calcular el error cuadrático medio en el conjunto de prueba
mse = mean_squared_error(y_test_mlp, y_pred_mlp)
rmse = np.sqrt(mse)
print(f'Raíz del Error Cuadrático Medio en el conjunto de prueba: {rmse}')

Vizualizar las Perdidas(Para detectar si hay Overfitting)

In [0]:
# Graficar las pérdidas de entrenamiento y validación
plt.plot(history.history['loss'], label='Pérdida de entrenamiento')
plt.plot(history.history['val_loss'], label='Pérdida de validación')
plt.title('Curva de aprendizaje')
plt.xlabel('Épocas')
plt.ylabel('Pérdida')
plt.legend()
plt.show()

Evaluar el rendimiento del Modelo en el Conj. de Entrenamiento y Validacion

In [0]:
# Hacer predicciones en el conjunto de ENTRENAMIENTO
y_train_pred_mlp = model_mlp.predict(x_train_mlp)

# Calcular el error cuadrático medio en el conjunto de entrenamiento
mse_train = mean_squared_error(y_train_mlp, y_train_pred_mlp)
rmse_train = np.sqrt(mse_train)
#----------------------------------------------------------------------------------------------------------------------------------------

# Hacer predicciones en el conjunto de VALIDACION
y_pred_val_mlp = model_mlp.predict(x_val_mlp)

# Calcular el error cuadrático medio en el conjunto de prueba
mse_val = mean_squared_error(y_val_mlp, y_pred_val_mlp)
rmse_val = np.sqrt(mse_val)

#--------------------------------------------------------------------------------------------------------------------------------------------
# Hacer predicciones en el conjunto de PRUEBA
y_pred_mlp = model_mlp.predict(x_test_mlp)

# Calcular el error cuadrático medio en el conjunto de prueba
mse_test = mean_squared_error(y_test_mlp, y_pred_mlp)
rmse_test = np.sqrt(mse_test)

print(f'Raíz del Error Cuadrático Medio en el conjunto de entrenamiento: {rmse_train}')
print(f'Raíz del Error Cuadrático Medio en el conjunto de entrenamiento: {rmse_val}')
print(f'Raíz del Error Cuadrático Medio en el conjunto de prueba: {rmse_test}')

#### Feature Importances in REDES NEURONALES

Metodo 1: Metodo Basado en Gradientes
- Gradiente: En términos matemáticos, el gradiente es una medida de cuánto cambia la salida de una función (en este caso, la red neuronal) en respuesta a cambios en sus entradas. Es un vector que apunta en la dirección del mayor aumento de la función.

- Importancia de las características: La importancia de una característica se puede estimar observando la magnitud del gradiente de la salida con respecto a esa característica. Una mayor magnitud del gradiente sugiere que pequeños cambios en esa característica tienen un gran impacto en la salida, lo que indica una mayor importancia.

- Calcular los gradientes: Una vez que la red neuronal está entrenada, se calculan los gradientes de la salida de la red con respecto a cada característica de entrada. Esto se hace usando el cálculo automático de derivadas, que es una capacidad estándar en bibliotecas de aprendizaje profundo como TensorFlow y PyTorch.

In [0]:
# model = tf.keras.models.load_model('your_model.h5') # Load your trained model

# X = np.load('your_input_data.npy') # Load your input data

# Convertir X(datos de caracteristicas MLP Red Neuronal Multicapa) a tensor de TensorFlow
input_tensor = tf.convert_to_tensor(X)

# Calcular gradientes de la salida con respecto a las entradas
with tf.GradientTape() as tape:
    tape.watch(input_tensor)
    output = model_mlp(input_tensor)

gradients = tape.gradient(output, input_tensor)

# Calcular la importancia de las características como la media absoluta de los gradientes
feature_importance = np.mean(np.abs(gradients.numpy()), axis=0)

# Obtener los nombres de las características del DataFrame original
feature_names = datos_total_escalados_mlp.columns

# Crear un DataFrame para asociar nombres de características con sus importancias
importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': feature_importance
})

# Ordenar las características por importancia
importance_df = importance_df.sort_values(by='Importance', ascending=False)
importance_df

Metodo 2:  basado en pesos (weight-based methods) 
- se utiliza para estimar la importancia de las características en una red neuronal considerando los pesos o conexiones entre las neuronas
- Extraer Pesos: Obtener los pesos de las capas del modelo, especialmente de la primera capa si se está evaluando la importancia de las características de entrada.
-Calcular Importancia: Calcular la importancia de cada característica sumando los valores absolutos de sus pesos. Esto se basa en la idea de que los pesos con valores absolutos mayores tienen un impacto mayor en la salida del modelo.

In [0]:
# Obtener los pesos de las capas del modelo
weights = model_mlp.get_weights()

# Calcular la importancia de las características como la suma de los valores absolutos de los pesos de la primera capa
feature_importance = np.sum(np.abs(weights[0]), axis=1)

# Obtener los nombres de las características del DataFrame original
feature_names = datos_total_escalados_mlp.columns

# Crear un DataFrame para asociar nombres de características con sus importancias
importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': feature_importance
})

# Ordenar las características por importancia
importance_df = importance_df.sort_values(by='Importance', ascending=False)

print("Importancia de las características:")
print(importance_df)

#### Convertir el Modelo a Tflite

1. Convertir desde la misma sesion y Configurar el convertidor TFLite

In [0]:
import tensorflow as tf

# Convertir desde la misma sesion y Configurar el convertidor TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(model=model_mlp)

# Especificar las operaciones admitidas por el modelo TFLite
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]

# Deshabilitar la conversión de operaciones de lista de tensores
converter._experimental_lower_tensor_list_ops = False

# Convertir el modelo a TensorFlow Lite
model_tflite = converter.convert()

# Guardar el modelo TensorFlow Lite en disco
with open("model_mlp_v1.tflite", "wb") as f:
    f.write(model_tflite)

2. Convertir desde Cargar el modelo desde la ubicacion donde se guardo con "SavedModel de TensorFlow"

 Guardar el modelo entrenado

In [0]:
# Guardar el modelo entrenado

# Guardando todos los componentes del modelo (arquitectura, pesos, configuración de entrenamiento, estado del optimizador, etc.)
# SavedModel de TensorFlow
model_mlp.save("model_mlp_v1.h5")

In [0]:
import tensorflow as tf

# Cargar el modelo desde la ubicacion donde se guardo con "SavedModel de TensorFlow"
model_cargado = tf.keras.models.load_model('model_mlp_v1.h5')

# Convertir el modelo a formato .tflite
converter2 = tf.lite.TFLiteConverter.from_keras_model(model_cargado)

# Especificar el conjunto de operaciones admitidas
converter2.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,  # Operaciones TensorFlow Lite estándar (mayoría de los modelos, como convoluciones, activaciones, etc.)
    tf.lite.OpsSet.SELECT_TF_OPS      # Operaciones TensorFlow adicionales - Esto debes instalar en el Android Studio Gradle
]

# Convertir el modelo
tflite_model2 = converter2.convert()

# Guardar el modelo convertido en formato .tflite
with open('model_mlp_v1_2.tflite', 'wb') as f:
    f.write(tflite_model2)

#### Tunnig de Hiperparametros del Modelo de AI

1.Metodo de Grid Search 

In [0]:
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error

# Suponiendo que tienes tus datos X e y definidos antes de esta sección

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train_rnn, X_test_rnn, y_train_rnn, y_test_rnn = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear una función para construir el modelo
def create_model(optimizer='adam'):
    model_rnn = Sequential()
    model_rnn.add(Dense(10, input_dim=5, activation='relu'))
    model_rnn.add(Dense(20, activation='relu'))
    model_rnn.add(Dense(10, activation='relu'))
    model_rnn.add(Dense(1, activation='linear'))
    model_rnn.compile(loss='mean_squared_error', optimizer=optimizer)
    return model_rnn

# Crear el KerasRegressor
model_rnn = KerasRegressor(build_fn=create_model, verbose=0)

# Definir la búsqueda en cuadrícula de hiperparámetros
param_grid = {
    'batch_size': [10, 20],
    'epochs': [20, 40, 60, 100],
    'optimizer': ['adam', 'rmsprop']
}

# Realizar la búsqueda en cuadrícula
grid = GridSearchCV(estimator=model_rnn, param_grid=param_grid, scoring='neg_mean_squared_error', cv=3)
grid_result = grid.fit(X_train_rnn, y_train_rnn)

# Resumir los resultados
print(f"Mejor puntuación (neg_mean_squared_error): {grid_result.best_score_}")
print(f"Mejores hiperparámetros: {grid_result.best_params_}")

# Obtener el mejor modelo
best_model = grid_result.best_estimator_

# Hacer predicciones en el conjunto de prueba
y_pred_rnn = best_model.predict(X_test_rnn)

# Calcular el error cuadrático medio en el conjunto de prueba
mse = mean_squared_error(y_test_rnn, y_pred_rnn)
# Calcular la raíz cuadrada del error cuadrático medio
rmse = np.sqrt(mse)

print(f'Raíz del Error Cuadrático Medio en el conjunto de prueba: {rmse}')

# Plot training & validation loss values
plt.plot(grid_result.best_estimator_.model.history.history['loss'])
plt.plot(grid_result.best_estimator_.model.history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()


#### 5.8 Red Neuronal LSTM

In [0]:
## Luego de Hacer el Feature Importances (Metodo de Envoltura) se identifico este conjunto de variables mas importantes
datos_total_escalados_lstm =  datos_total_escalados[['tonelaje_inicial_poligono_creado',
'elevacion_poligono_metros',
'CF02','CF01', 'CF03', 'PH03', 'PH02', 'PH01', 'FR01',
'tiempo_ready_llegada_esperando',
'cantidad_equipos_espera_al_termino_carga_pala',
'densidad_inicial_poligono_creado_tn/m3']]

# Supongamos que datos es tu DataFrame y has seleccionado tus características (X) y variable objetivo (y)
# Capacidad_peso = capacidad_volumen * densidad   O  capacidad_volumen = Capacidad_peso / densidad
X = datos_total_escalados_lstm.values #Sale de los datos escalados

y = datos['numero_pases_carguio'].values # Reemplaza 'variable_objetivo' con el nombre de tu variable objetivo(sale de los datos originales)

In [0]:
X.shape[1]

In [0]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np
import matplotlib.pyplot as plt

# En LSTM, debes mantener la secuencia al dividir en Train, Validation, Test

# Definir los porcentajes para cada conjunto (ajustados según tu necesidad)
train_ratio = 0.75  # 75% para entrenamiento
validation_ratio = 0.15  # 15% para validación
test_ratio = 0.10  # 10% para prueba

# Calcular los índices para dividir los datos
num_samples = len(X)
train_idx = int(train_ratio * num_samples)
val_idx = int((train_ratio + validation_ratio) * num_samples)

# Dividir los datos en conjuntos de entrenamiento, validación y prueba
x_train_lstm, y_train_lstm = X[:train_idx], y[:train_idx]
x_val_lstm, y_val_lstm = X[train_idx:val_idx], y[train_idx:val_idx]
x_test_lstm, y_test_lstm = X[val_idx:], y[val_idx:]


# Crear el modelo LSTM
model_lstm = Sequential()
#Cada unidad LSTM calcula y actualiza su estado interno basado en la entrada actual, el estado anterior y una compuerta de olvido que controla cuánta #información pasa a través del tiempo.
model_lstm.add(LSTM(30, activation='relu', input_shape=(1, X.shape[1]))) # X.shape[1] : Numero de variables input
model_lstm.add(Dense(50, activation='relu'))  # ReLU(x)=max(0,x)
#ReLU introduce no linealidad en el modelo, Sin esta no linealidad, las redes neuronales se reducirían a combinaciones lineales de sus entradas
# ReLU Al permitir que las activaciones positivas pasen sin cambio y descartar las negativas, ReLU facilita que la red neuronal aprenda representaciones más #discriminativas de los datos
model_lstm.add(Dense(30, activation='relu'))
model_lstm.add(Dense(15, activation='relu'))
model_lstm.add(Dense(1, activation='linear'))

# Compilar el modelo
model_lstm.compile(loss='mean_squared_error', optimizer='adam')
model_lstm.summary()

# Entrenar el modelo y registrar las pérdidas input:(numero_muestras, bacthes, num_variables), output:(numero_muestras, bacthes, num_variables)
history = model_lstm.fit(x_train_lstm.reshape(-1, 1, X.shape[1]), y_train_lstm, epochs=50, batch_size=5, verbose=2, validation_data=(x_val_lstm.reshape(-1, 1, X.shape[1]), y_val_lstm))

# Hacer predicciones en el conjunto de prueba
y_pred_lstm = model_lstm.predict(x_test_lstm.reshape(-1, 1, X.shape[1]))
y_pred_lstm = np.round(y_pred_lstm).astype('int64')

# Calcular el error cuadrático medio en el conjunto de prueba
mse = mean_squared_error(y_test_lstm, y_pred_lstm)
rmse = np.sqrt(mse)
print(f'Raíz del Error Cuadrático Medio en el conjunto de prueba: {rmse}')

Vizualizar las Perdidas(Para detectar si hay Overfitting)

In [0]:
# Graficar las pérdidas de entrenamiento y validación
plt.plot(history.history['loss'], label='Pérdida de entrenamiento')
plt.plot(history.history['val_loss'], label='Pérdida de validación')
plt.title('Curva de aprendizaje LSTM')
plt.xlabel('Épocas')
plt.ylabel('Pérdida')
plt.legend()
plt.show()

Evaluar el rendimiento del Modelo en el Conj. de Entrenamiento y Validacion

In [0]:
y_test_lstm.shape

In [0]:
# Hacer predicciones en el conjunto de ENTRENAMIENTO
y_train_pred_lstm = model_lstm.predict(x_train_lstm.reshape(-1, 1, X.shape[1]))

# Calcular el error cuadrático medio en el conjunto de entrenamiento
mse_train_lstm = mean_squared_error(y_train_lstm, y_train_pred_lstm)
rmse_train_lstm = np.sqrt(mse_train_lstm)
#----------------------------------------------------------------------------------------------------------------------------------------

# # Hacer predicciones en el conjunto de VALIDACION
y_val_pred_lstm = model_lstm.predict(x_val_lstm.reshape(-1, 1, X.shape[1]))

# # Calcular el error cuadrático medio en el conjunto de prueba
mse_val_lstm = mean_squared_error(y_val_lstm, y_val_pred_lstm)
rmse_val_lstm = np.sqrt(mse_val_lstm)

#--------------------------------------------------------------------------------------------------------------------------------------------
# Hacer predicciones en el conjunto de PRUEBA
y_test_pred_lstm = model_lstm.predict(x_test_lstm.reshape(-1, 1, X.shape[1]))

# Calcular el error cuadrático medio en el conjunto de prueba
mse_test_lstm = mean_squared_error(y_test_lstm, y_test_pred_lstm)
rmse_test_lstm = np.sqrt(mse_test_lstm)

print(f'Raíz del Error Cuadrático Medio en el conjunto de entrenamiento: {rmse_train_lstm}')
print(f'Raíz del Error Cuadrático Medio en el conjunto de entrenamiento: {rmse_val_lstm}')
print(f'Raíz del Error Cuadrático Medio en el conjunto de prueba: {rmse_test_lstm}')

#### Feature Importances in REDES NEURONALES

Metodo 1: Metodo Basado en Gradientes
- Gradiente: En términos matemáticos, el gradiente es una medida de cuánto cambia la salida de una función (en este caso, la red neuronal) en respuesta a cambios en sus entradas. Es un vector que apunta en la dirección del mayor aumento de la función.

- Importancia de las características: La importancia de una característica se puede estimar observando la magnitud del gradiente de la salida con respecto a esa característica. Una mayor magnitud del gradiente sugiere que pequeños cambios en esa característica tienen un gran impacto en la salida, lo que indica una mayor importancia.

- Calcular los gradientes: Una vez que la red neuronal está entrenada, se calculan los gradientes de la salida de la red con respecto a cada característica de entrada. Esto se hace usando el cálculo automático de derivadas, que es una capacidad estándar en bibliotecas de aprendizaje profundo como TensorFlow y PyTorch.

In [0]:
# model = tf.keras.models.load_model('your_model.h5') # Load your trained model

# X = np.load('your_input_data.npy') # Load your input data

# Expandir la dimensión de los datos de entrada para que sean compatibles con el modelo LSTM
#X_expanded = np.expand_dims(X, axis=1)  # (330118, 26) -> (330118, 1, 26)
X_reshape = X.reshape(-1, 1, X.shape[1])       # (330118, 26) -> (330118, 1, 26), X.shape[1] : Numero variables del modelo
# X_reshape.shape

# Convertir X(datos de caracteristicas LSTM) a tensor de TensorFlow
input_tensor = tf.convert_to_tensor(X_reshape)

# Calcular gradientes de la salida con respecto a las entradas
with tf.GradientTape() as tape:
    tape.watch(input_tensor)
    output = model_lstm(input_tensor)

gradients = tape.gradient(output, input_tensor)

# Reducir la dimensión del tensor de gradientes
gradients = tf.squeeze(gradients, axis=1)  # (330118, 1, 26) -> (330118, 26)

# Calcular la importancia de las características como la media absoluta de los gradientes
feature_importance = np.mean(np.abs(gradients.numpy()), axis=0)

# Obtener los nombres de las características del DataFrame original
feature_names = datos_total_escalados_lstm.columns

# Crear un DataFrame para asociar nombres de características con sus importancias
importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': feature_importance
})

# Ordenar las características por importancia
importance_df = importance_df.sort_values(by='Importance', ascending=False)
importance_df

Metodo 2:  basado en pesos (weight-based methods) 
- se utiliza para estimar la importancia de las características en una red neuronal considerando los pesos o conexiones entre las neuronas
- Extraer Pesos: Obtener los pesos de las capas del modelo, especialmente de la primera capa si se está evaluando la importancia de las características de entrada.
-Calcular Importancia: Calcular la importancia de cada característica sumando los valores absolutos de sus pesos. Esto se basa en la idea de que los pesos con valores absolutos mayores tienen un impacto mayor en la salida del modelo.

In [0]:
# Obtener los pesos de las capas del modelo
weights = model_lstm.get_weights()

# Calcular la importancia de las características como la suma de los valores absolutos de los pesos de la primera capa
feature_importance = np.sum(np.abs(weights[0]), axis=1)

# Obtener los nombres de las características del DataFrame original
feature_names = datos_total_escalados_lstm.columns

# Crear un DataFrame para asociar nombres de características con sus importancias
importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': feature_importance
})

# Ordenar las características por importancia
importance_df = importance_df.sort_values(by='Importance', ascending=False)

print("Importancia de las características:")
print(importance_df)

Metodo 3: Extracción y Procesamiento de Pesos de Todas las Capas

In [0]:
import tensorflow as tf
import numpy as np
import pandas as pd

# # Cargar el modelo previamente entrenado
# model = tf.keras.models.load_model('your_model.h5')

# Obtener los pesos de las capas del modelo
weights = model_rnn.get_weights()

# Inicializar una lista para almacenar la importancia de cada característica
num_features = weights[0].shape[0]
total_importance = np.zeros(num_features)

# Calcular la importancia de cada característica a través de todas las capas
for i in range(0, len(weights), 2):  # Saltamos cada 2 para solo tomar los pesos y no los biases
    layer_weights = weights[i]
    if len(layer_weights.shape) == 2:  # Asegurar que es una capa densamente conectada
        layer_importance = np.sum(np.abs(layer_weights), axis=1)
        total_importance += layer_importance

# Obtener los nombres de las características del DataFrame original
feature_names = datos_total_escalados.columns

# Crear un DataFrame para asociar nombres de características con sus importancias
importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': total_importance
})

# Ordenar las características por importancia
importance_df = importance_df.sort_values(by='Importance', ascending=False)

print("Importancia de las características:")
print(importance_df) 

#### Convertir el Modelo a Tflite

1. Convertir desde la misma sesion y Configurar el convertidor TFLite

In [0]:
import tensorflow as tf

# Convertir desde la misma sesion y Configurar el convertidor TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(model=model_lstm)

# Especificar las operaciones admitidas por el modelo TFLite
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]

# Deshabilitar la conversión de operaciones de lista de tensores
#(converter._experimental_lower_tensor_list_ops = True): Significa que el convertidor intentará simplificar las operaciones de listas de tensores a operaciones más primitivas y compatibles con TensorFlow Lite (TFLite).
#(converter._experimental_lower_tensor_list_ops = False): Significa que el convertidor no realizará esta simplificación, y las operaciones de listas de tensores se mantendrán en su forma original, lo cual puede resultar en incompatibilidades con TFLite.
converter._experimental_lower_tensor_list_ops = False

# Convertir el modelo a TensorFlow Lite
model_tflite = converter.convert()

# Guardar el modelo TensorFlow Lite en disco
with open("model_lstm_v1.tflite", "wb") as f:
    f.write(model_tflite)

2. Convertir desde Cargar el modelo desde la ubicacion donde se guardo con "SavedModel de TensorFlow"

 Guardar el modelo entrenado

In [0]:
# Guardar el modelo entrenado

# Guardando todos los componentes del modelo (arquitectura, pesos, configuración de entrenamiento, estado del optimizador, etc.)
# SavedModel de TensorFlow
model_lstm.save("model_lstm_v1.h5")

In [0]:
import tensorflow as tf

# Cargar el modelo desde la ubicacion donde se guardo con "SavedModel de TensorFlow"
model_cargado = tf.keras.models.load_model('model_mlp_v1.h5')

# Convertir el modelo a formato .tflite
converter2 = tf.lite.TFLiteConverter.from_keras_model(model_cargado)

# Especificar el conjunto de operaciones admitidas
converter2.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,  # Operaciones TensorFlow Lite estándar (mayoría de los modelos, como convoluciones, activaciones, etc.)
    tf.lite.OpsSet.SELECT_TF_OPS      # Operaciones TensorFlow adicionales - Esto debes instalar en el Android Studio Gradle
]

#(converter._experimental_lower_tensor_list_ops = True): Significa que el convertidor intentará simplificar las operaciones de listas de tensores a operaciones más primitivas y compatibles con TensorFlow Lite (TFLite).
#(converter._experimental_lower_tensor_list_ops = False): Significa que el convertidor no realizará esta simplificación, y las operaciones de listas de tensores se mantendrán en su forma original, lo cual puede resultar en incompatibilidades con TFLite.
converter._experimental_lower_tensor_list_ops = False

# Convertir el modelo
tflite_model2 = converter2.convert()

# Guardar el modelo convertido en formato .tflite
with open('model_mlp_v1_2.tflite', 'wb') as f:
    f.write(tflite_model2)

In [0]:
F

#### 6. Armando los BaseFinal, con los valores predichos de cada Modelo, y tambien medir el impacto con productividades

In [0]:
# Agregar las predicciones como una columna adicional al conjunto de prueba
df_resultados_models = combined_df.copy()
df_resultados_models['Numero_pases_real'] = y_test_gen
df_resultados_models['Numero_pases_reglineal'] = numero_pases_rl
df_resultados_models['Numero_pases_arboldecision'] = numero_pases_dt
df_resultados_models['Numero_pases_rforest'] = numero_pases_rf
df_resultados_models['Numero_pases_xgb'] = numero_pases_xgb
df_resultados_models['Numero_pases_mlp'] = numero_pases_mlp
df_resultados_models['Numero_pases_lstm'] = numero_pases_lstm
df_resultados_models['capacidad_en_peso_equipo_carguio'] = datos['capacidad_en_peso_equipo_carguio']
df_resultados_models['tonelaje_vims_all'] = datos['tonelaje_segun_computadora']
df_resultados_models.head()
#------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

#-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
#Regresion Lineal
#df_resultados_models['Diferencia_Pases_rl'] = df_resultados_models['Numero_pases_reglineal'] - df_resultados_models['numero_pases_carguio']
df_resultados_models['tonelaje_segun_computadora_rl'] = (df_resultados_models['tonelaje_vims_all']) + (df_resultados_models['Diferencia_Pases_rl']*df_resultados_models['capacidad_en_peso_equipo_carguio'])


#Arbol de decision
df_resultados_models['Diferencia_Pases_dt'] = df_resultados_models['Numero_pases_arboldecision'] - df_resultados_models['numero_pases_carguio']
df_resultados_models['tonelaje_segun_computadora_dt'] = (df_resultados_models['tonelaje_vims_all']) + (df_resultados_models['Diferencia_Pases_dt']*df_resultados_models['capacidad_en_peso_equipo_carguio'])


# Random Forest 
df_resultados_models['Diferencia_Pases_rf'] = df_resultados_models['Numero_pases_rforest'] - df_resultados_models['numero_pases_carguio']
df_resultados_models['tonelaje_segun_computadora_rf'] = (df_resultados_models['tonelaje_vims_all']) + (df_resultados_models['Diferencia_Pases_rf']*df_resultados_models['capacidad_en_peso_equipo_carguio'])


# XGB (eXtreme Gradient Boosting)
df_resultados_models['Diferencia_Pases_xgb'] = df_resultados_models['Numero_pases_xgb'] - df_resultados_models['numero_pases_carguio']
df_resultados_models['tonelaje_segun_computadora_xgb'] = (df_resultados_models['tonelaje_vims_all']) + (df_resultados_models['Diferencia_Pases_xgb']*df_resultados_models['capacidad_en_peso_equipo_carguio'])

# RNN (Redes Neuronales MLP)
df_resultados_models['Diferencia_Pases_MLP'] = df_resultados_models['Numero_pases_mlp'] - df_resultados_models['numero_pases_carguio']
df_resultados_models['tonelaje_segun_computadora_MLP'] = (df_resultados_models['tonelaje_vims_all']) + (df_resultados_models['Diferencia_Pases_MLP']*df_resultados_models['capacidad_en_peso_equipo_carguio'])

# RNN (Redes Neuronales LSTM)
df_resultados_models['Diferencia_Pases_LSTM'] = df_resultados_models['Numero_pases_lstm'] - df_resultados_models['numero_pases_carguio']
df_resultados_models['tonelaje_segun_computadora_LSTM'] = (df_resultados_models['tonelaje_vims_all']) + (df_resultados_models['Diferencia_Pases_LSTM']*df_resultados_models['capacidad_en_peso_equipo_carguio'])


df_resultados_models.head(10)

## Part 1. Train a classification model

### MLflow Tracking
[MLflow tracking](https://www.mlflow.org/docs/latest/tracking.html) allows you to organize your machine learning training code, parameters, and models. 

You can enable automatic MLflow tracking by using [*autologging*](https://www.mlflow.org/docs/latest/tracking.html#automatic-logging).

In [0]:
# Establecer semillas en NumPy y TensorFlow
np.random.seed(42)
tf.random.set_seed(42)

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train_rnn, X_test_rnn, y_train_rnn, y_test_rnn = train_test_split(X, y, test_size=0.2, random_state=42)

Modelo 1 - Redes LSTM

In [0]:
#Librerias para redes neuronales(LSTM)
from tensorflow.keras.layers import Dense, LSTM
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Normalization
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

# Establecer semillas en NumPy y TensorFlow
np.random.seed(42)
tf.random.set_seed(42)

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train_rnn, X_test_rnn, y_train_rnn, y_test_rnn = train_test_split(X, y, test_size=0.2, random_state=42)
#X_train_rnn, X_test_rnn, y_train_rnn, y_test_rnn = train_test_split(X_array, y_array, test_size=0.2, random_state=42)

model_rnn = Sequential()
# La forma de entrada para LSTM debe ser (n_timesteps, n_features)
model_rnn.add(LSTM(30, activation='relu', input_shape=(1, 5)))  #  (n_samples, n_pasos, n_variables)
# Agregar capas Dense según sea necesario
model_rnn.add(Dense(60, activation='relu'))
model_rnn.add(Dense(30, activation='relu'))
model_rnn.add(Dense(15, activation='relu'))
# Capa de salida
model_rnn.add(Dense(1, activation='linear'))
#-----------------------------------------------------------------------------------------------------------------------------------------

# Compilar el modelo
model_rnn.compile(loss='mean_squared_error', optimizer='adam')
# Imprimir un resumen del modelo
model_rnn.summary()

# Entrenar el modelo
history = model_rnn.fit(X_train_rnn.reshape(-1, 1, 5), y_train_rnn, epochs=1, validation_data=(X_test_rnn.reshape(-1, 1, 5),y_test_rnn), batch_size=5, verbose=1)

# VALIDACION 1
# Obtener la pérdida en el conjunto de entrenamiento y el conjunto de validación
train_loss = history.history['loss']
val_loss = history.history['val_loss']

# Calcular el RMSE en el conjunto de entrenamiento y el conjunto de validación
train_rmse = np.sqrt(train_loss)
val_rmse = np.sqrt(val_loss)

# VALIDACION 2
# Hacer predicciones en el conjunto de prueba
y_pred_rnn = model_rnn.predict(X_test_rnn.reshape(-1, 1, 5)) # (n_samples, n_pasos, n_variables)

# Redondear los valores de y_pred al entero más cercano
y_pred_rnn = np.round(y_pred_rnn).astype('int64')

# Calcular el error cuadrático medio en el conjunto de prueba
mse = mean_squared_error(y_test_rnn, y_pred_rnn)
# Calcular la raíz cuadrada del error cuadrático medio
rmse = np.sqrt(mse)
print(f'Raíz del Error Cuadrático Medio en el conjunto de prueba: {rmse}')

In [0]:
import mlflow
import mlflow.keras
import numpy as np
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

# Establecer semillas en NumPy y TensorFlow
np.random.seed(42)
tf.random.set_seed(42)

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train_rnn, X_test_rnn, y_train_rnn, y_test_rnn = train_test_split(X, y, test_size=0.2, random_state=42)

#MLflow podra registrar automáticamente métricas, parámetros y el modelo generado durante el entrenamiento
mlflow.tensorflow.autolog()  # Keras y TensorFlow

# Entrenar el modelo
with mlflow.start_run(run_name='experimento_MLOPS_V1'):  #Iniciliza un nuevo experimento de MlFlow
    # Definir y compilar el modelo
    model_rnn = Sequential()
    model_rnn.add(LSTM(30, activation='relu', input_shape=(1, 5)))
    model_rnn.add(Dense(60, activation='relu'))
    model_rnn.add(Dense(30, activation='relu'))
    model_rnn.add(Dense(15, activation='relu'))
    model_rnn.add(Dense(1, activation='linear'))

    model_rnn.compile(loss='mean_squared_error', optimizer='adam')

    # Entrenar el modelo y guardar el historial del entrenamiento
    history = model_rnn.fit(X_train_rnn.reshape(-1, 1, 5), y_train_rnn, epochs=1, validation_data=(X_test_rnn.reshape(-1, 1, 5), y_test_rnn), batch_size=5, verbose=1)

    # Save the run information to register the model later
    #kerasURI = run.info.artifact_uri

    # Registrar los parámetros del modelo
    mlflow.log_params({
        "input_shape": (1, 5),
        "lstm_units": 30,
        "dense_layers": [60, 30, 15],
        "activation": "relu",
        "optimizer": "adam",
        "loss_function": "mean_squared_error",
        "epochs": 1,
        "batch_size": 5
    })

    # Registrar la estructura del modelo en MLflow
    #mlflow.keras.log_model(model_rnn, "keras_tf_v1")

    # VALIDACION 
    # Obtener la pérdida en el conjunto de entrenamiento y el conjunto de validación
    train_loss = history.history['loss']
    test_loss = history.history['val_loss']

    # Calcular el RMSE en el conjunto de entrenamiento y el conjunto de validación
    train_rmse = np.sqrt(train_loss)
    test_rmse = np.sqrt(test_loss)

    # Registro de métricas de RMSE
    mlflow.log_metric("train_RMSE", train_rmse[-1])  # Se registra el último valor de RMSE del conjunto de entrenamiento
    mlflow.log_metric("test_RMSE", test_rmse[-1])      # Se registra el último valor de RMSE del conjunto de validación
    
    # Crear gráfico de pérdida durante el entrenamiento
    epochs = range(1, len(train_loss) + 1)
    plt.plot(epochs, train_loss, 'bo', label='Pérdida en entrenamiento')
    plt.plot(epochs, test_loss, 'r', label='Pérdida en validación')
    plt.title('Pérdida durante el entrenamiento y la validación')
    plt.xlabel('Épocas')
    plt.ylabel('Pérdida')
    plt.legend()
    plt.savefig("kerasplotv3.png")
    mlflow.log_artifact("kerasplotv3.png")

In [0]:
model_path

#### Register to Model Registry

In [0]:
model_name = "Example_rnn"                                  #nombre que se le dara al modelo que registraremos en el MlFlow Registry
model_uri = f"runs:/ed91e81d7ca644728f3e223fa3cfac92/model" #identificador unico del experimento a registrar en Mlflow
registered_model_version = mlflow.register_model(model_uri, model_name)

#### Load from Model Registry 

In [0]:
import numpy as np
import mlflow

client = mlflow.tracking.MlflowClient() 
model_name = "Example_rnn"

model_info = client.get_registered_model(model_name)
model_version = model_info.latest_versions[0].version #Ultima version del modelo en MlFlow Registry
#model_version = registered_model_version.version

model = mlflow.pyfunc.load_model(model_uri=f"models:/{model_name}/{model_version}")

# Supongamos que tienes tus datos de entrada en una lista
input_data = np.array([[12, 35, 129, 48, 129]])

# # Convertir los datos de entrada al tipo de datos float64
input_data = input_data.astype(np.float64)

# # Aumentamos a 3dimensiones los datos de entrada para que tengan la forma (None, 1, 5)
input_data = np.reshape(input_data, (input_data.shape[0], 1, input_data.shape[1]))

# # Realiza la predicción utilizando el modelo
model.predict(input_data)

### Load model without experiments

In [0]:
model_uri = f"runs:/ed91e81d7ca644728f3e223fa3cfac92/model"
model = mlflow.pyfunc.load_model(model_uri)

#### Actualizar el modelo y tener una nueva version del Modelo de MlFlow Registry

In [0]:
import mlflow

# Definir el nombre del modelo y la nueva URI del modelo (para la versión 2)
model_name = "Example_rnn"
new_model_uri = 'runs:/3fdcef4bd83943ec9dc2188bc13045f4/model'  # Ejemplo de nueva URI del modelo

# Registrar la nueva versión del modelo con la misma nombre pero con una nueva URI
registered_model_version = mlflow.register_model(new_model_uri, model_name)

# Si deseas, puedes obtener el ID de la nueva versión del modelo registrado
new_version_id = registered_model_version.version
new_version_id

#### Validacion del Modelo antes de pasasr a Staging

In [0]:
#Obtener el modelo  in trancision, su nombre y version por el Webbook
#model_name, model_version = fech_webbook_data()
import mlflow
# Interactuar con el servidor de MLflow
client = mlflow.tracking.MlflowClient()  
model_name = "Example_rnn"

model_info = client.get_registered_model(model_name)
model_version = model_info.latest_versions[0].version #Ultima version del modelo en MlFlow Registry

models_details = client.get_model_version(model_name, model_version)
run_info = client.get_run(run_id=models_details.run_id)

In [0]:
run_info

#### Validacion de la prediccion

In [0]:
from databricks.feature_store import FeatureStoreClient

In [0]:
from databricks.feature_store import FeatureStoreClient

fs = FeatureStoreClient()

#Leer de Feature Store 
data_source = run_info.data.tags['db_table']
features = fs.read_table(data_source)

#Load model as a spark UDF
model_uri = f"models:/{model_name}/{model_version}"
loaded_model = mlflow.pyfunc.spark_udf(spark, model_uri=model_uri)

#Selecciona las columnas de la tabla de caracteristicas, por el esquema input del modelo
input_columns_names = loaded_model.metadata.get_input_schema().input_names()

#Predict in spark dataframe
try:
    display(features.withColumn('predictions', loaded_model(*input_columns_names)))
    client.set_model_version_tag(name=model_name, version=model_version, key="predicts", value=1)
except Exception:
    print("No se puede predecir las caracteristicas,")
    client.set_model_version_tag(name=model_name, version=model_version, key="predicts", value=0)
    pass


In [0]:
import mlflow

# Nombre del modelo registrado
model_name = "MLops_kerastf_v3"

# Crear una instancia del cliente de MLflow
client = mlflow.tracking.MlflowClient()

# Obtener la información sobre el modelo
model_info = client.get_registered_model(model_name)

In [0]:
model_info

In [0]:
# Estado al que deseas cambiar el modelo (p. ej., "Production", "Staging", "Archived")
new_stage = "None"

# Versión específica que deseas cambiar de estado
#target_version = model_info.latest_versions[0].version # ultima version del modelo registrado en mlflow registry
target_version = 1


# Cambiar el estado del modelo
client.transition_model_version_stage(
    name=model_name,
    version = target_version,  #
    stage=new_stage
)


In [0]:
# Estado al que deseas cambiar el modelo (p. ej., "Production", "Staging", "Archived")
new_stage = "Production"

# Cambiar el estado del modelo
client.transition_model_version_stage(
    name=model_name,
    version=model_info.latest_versions[0].version,
    stage=new_stage
)

In [0]:
import matplotlib.pyplot as plt
# Crear gráfico de pérdida durante el entrenamiento
epochs = range(1, len(train_loss) + 1)
plt.plot(epochs, train_loss, 'bo', label='Pérdida en entrenamiento')
plt.plot(epochs, val_loss, 'r', label='Pérdida en validación')
plt.title('Pérdida durante el entrenamiento y la validación')
plt.xlabel('Épocas')
plt.ylabel('Pérdida')
plt.legend()
plt.show()

Modelo 2 : Redes Tradicionales

In [0]:
#Librerias para redes neuronales(LSTM)
from tensorflow.keras.layers import Dense, LSTM
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Normalization
from tensorflow.keras.optimizers import Adam
#from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score

# 3. Construccion del modelo de RED NEURONAL
# Aseguramos que los resultados sean "reproducibles" en cada ejecucion de tensorflow(pesos iniciales aleatorios)
np.random.seed(42)
tf.random.set_seed(42)

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train_rnn, X_test_rnn, y_train_rnn, y_test_rnn = train_test_split(X, y, test_size=0.2, random_state=42)

#Arquitectura 2:
model_rnn = Sequential()
model_rnn.add(Dense(15, activation='relu', input_shape=[5]))
model_rnn.add(Dense(7, activation='relu'))
model_rnn.add(Dense(1, activation='linear'))

# Compilar el modelo
model_rnn.compile(loss='mean_squared_error', optimizer='adam')
# Imprimir un resumen del modelo
model_rnn.summary()

# Entrenar el modelo
# Red LSTM
#history = model_rnn.fit(X_train_rnn.reshape(-1, 1, 5), y_train_rnn, epochs=5, validation_data=(X_test_rnn.reshape(-1, 1, 5),y_test_rnn), batch_size=5, verbose=1)
# Red Tradicional
history = model_rnn.fit(X_train_rnn, y_train_rnn, epochs=4, validation_data=(X_test_rnn,y_test_rnn), batch_size=5, verbose=1)

# 4. Validacion del Modelo con la metrica de RSME
# VALIDACION 1 del Modelo
# Obtener la pérdida en el conjunto de entrenamiento y el conjunto de validación
train_loss = history.history['loss']
val_loss = history.history['val_loss']

# Calcular el RMSE en el conjunto de entrenamiento y el conjunto de validación
train_rmse = np.sqrt(train_loss)
val_rmse = np.sqrt(val_loss)

# VALIDACION 2 del Modelo
# Hacer predicciones en el conjunto de prueba Red LSTM
#y_pred_rnn = model_rnn.predict(X_test_rnn.reshape(-1, 1, 5)) # (n_samples, n_pasos, n_variables)
# Hacer predicciones en el conjunto de prueba Red Tradicional
y_pred_rnn = model_rnn.predict(X_test_rnn) # (n_samples, n_pasos, n_variables)

# Redondear los valores de y_pred al entero más cercano
y_pred_rnn = np.round(y_pred_rnn).astype('int64')

# Calcular el error cuadrático medio en el conjunto de prueba
mse = mean_squared_error(y_test_rnn, y_pred_rnn)
# Calcular la raíz cuadrada del error cuadrático medio
rmse = np.sqrt(mse)
print(f'Raíz del Error Cuadrático Medio en el conjunto de prueba: {rmse}')

In [0]:
import matplotlib.pyplot as plt
# Crear gráfico de pérdida durante el entrenamiento
epochs = range(1, len(train_loss) + 1)
plt.plot(epochs, train_loss, 'bo', label='Pérdida en entrenamiento')
plt.plot(epochs, val_loss, 'r', label='Pérdida en validación')
plt.title('Pérdida durante el entrenamiento y la validación')
plt.xlabel('Épocas')
plt.ylabel('Pérdida')
plt.legend()
plt.show()

In [0]:
np.unique(y_pred_rnn)

In [0]:
# Registrar un modelo en MLflow
model_path = "dbfs:/databricks/mlflow-tracking/8323b80e926241ee8763b7dc43085602/3fdcef4bd83943ec9dc2188bc13045f4/artifacts/model"
model_name = "MLops_kerastf_v1"
mlflow.register_model(model_path, model_name)

In [0]:
#ver el directorio local de databrinks mas no el local de .os
dbutils.fs.ls("dbfs:/")

### Guardar Modelo en Azure Storage

In [0]:
#1. Guardar en el directorio LOCAL (.OS)
model_rnn.save('/nuevomodelo/modelrnn_nuevo')

In [0]:
#2. Verificar que se ha guardado en el directorio LOCAL (.OS)
import os
os.listdir("/modelornn_version1_d4m")

Opcion 1: Cargar el Modelo ML directamente del Directorio (.OS)

In [0]:
# Cargar el modelo desde el contenedor de almacenamiento Azure
loaded_model_nuevo = tf.keras.models.load_model("/nuevomodelo/modelrnn_nuevo")

Opcion  2: Copiar el Modelo ML desde el Directorio LOCAL(.OS)  HASTA  Directorio de DBFS de Azure DataBrinks

In [0]:
#copie los datos desde LOCAL(.OS) a DBFS como dbfs (DIRECTORIO DE DATABRINKS)
dbutils.fs.cp("file:/nuevomodelo/modelrnn_nuevo", "dbfs:/nuevomodelo/modelrnn_nuevo", recurse=True) # True: si copias una carpeta, False: si #solo es archivo

Verificar que se ha migrado desde Directorio LOCAL(.OS)  Hacia Directorio DBFS

In [0]:
display(dbutils.fs.ls("/nuevomodelo/modelrnn_nuevo"))

Copiar el modelo desde el directorio DBFS al contenedor de almacenamiento Azure Storage

In [0]:
# Copiar el modelo desde el directorio DBFS al contenedor de almacenamiento Azure Storage
dbutils.fs.cp("/nuevomodelo/modelrnn_nuevo", "/mnt/datalakemlopsd4m/presentation/modelrnn_nuevo", recurse=True) #True:Copycarpeta,#False:Copyarchivo

In [0]:
import pandas as pd
import mlflow
import mlflow.keras
import mlflow.tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

# # Establecer la URI de seguimiento de MLflow
# mlflow.set_tracking_uri("databricks")  # Usa "databricks" para Databricks

# # Crear un nuevo experimento si no existe
# experiment_name = "nombre_del_experimento"
# mlflow.set_experiment(experiment_name)

# Cargar los datos
df_delta = spark.read.format("delta").load("/mnt/datalakemlopsd4m/presentation/datmarcobre_fengineer_tablacaract_delta")
datos = df_delta.toPandas()

# Separar las variables independientes y dependiente
X = datos[['capacidad_en_volumen_equipo_carguio_m3',
           'capacidad_en_peso_equipo_carguio',
           'capacidad_en_peso_equipo_acarreo',
           'densidad_inicial_poligono_creado_tn/m3',
           'capacidad_en_volumen_equipo_acarreo_m3']].values
y = datos['numero_pases_carguio'].values

# Construir el modelo de red neuronal
np.random.seed(42)
tf.random.set_seed(42)

X_train_rnn, X_test_rnn, y_train_rnn, y_test_rnn = train_test_split(X, y, test_size=0.2, random_state=42)

mlflow.tensorflow.autolog()  # Keras y TensorFlow

# Entrenar el modelo
with mlflow.start_run(run_name='experimento_mlflow_keras_tf_3_21_24'):  #Iniciliza un nuevo experimento de MlFlow
    model_rnn = Sequential()
    model_rnn.add(LSTM(30, activation='relu', input_shape=(1, 5)))
    model_rnn.add(Dense(60, activation='relu'))
    model_rnn.add(Dense(30, activation='relu'))
    model_rnn.add(Dense(15, activation='relu'))
    model_rnn.add(Dense(1, activation='linear'))

    model_rnn.compile(loss='mean_squared_error', optimizer='adam')

    history = model_rnn.fit(X_train_rnn.reshape(-1, 1, 5), y_train_rnn, epochs=1, 
                            validation_data=(X_test_rnn.reshape(-1, 1, 5), y_test_rnn), 
                            batch_size=5, verbose=1)

    # Registrar los parámetros del modelo
    mlflow.log_params({
        "input_shape": (1, 5),
        "lstm_units": 30,
        "dense_layers": [60, 30, 15],
        "activation": "relu",
        "optimizer": "adam",
        "loss_function": "mean_squared_error",
        "epochs": 1,
        "batch_size": 5
    })

    # Registrar la estructura del modelo en MLflow
    mlflow.keras.log_model(model_rnn, "keras_tf_3_21_24")

    # Calcular métricas
    train_loss = history.history['loss']
    test_loss = history.history['val_loss']
    train_rmse = np.sqrt(train_loss)
    test_rmse = np.sqrt(test_loss)

    # Registrar métricas de RMSE
    mlflow.log_metric("train_RMSE", train_rmse[-1])
    mlflow.log_metric("test_RMSE", test_rmse[-1])

    # Crear gráfico de pérdida durante el entrenamiento
    epochs = range(1, len(train_loss) + 1)
    plt.plot(epochs, train_loss, 'bo', label='Pérdida en entrenamiento')
    plt.plot(epochs, test_loss, 'r', label='Pérdida en validación')
    plt.title('Pérdida durante el entrenamiento y la validación')
    plt.xlabel('Épocas')
    plt.ylabel('Pérdida')
    plt.legend()
    plt.savefig("kerasplot_3_21_24.png")
    mlflow.log_artifact("kerasplot_3_21_24.png")

    # Obtener la ruta del modelo registrado
    model_path = mlflow.get_artifact_uri("keras_tf_3_21_24")

# Registrar el modelo en MLflow
model_name = "MLops_kerastf_3_21_24"
mlflow.register_model(model_path, model_name)